# Проект 8: "*Возьмете Bat-мобиль?*".
## Группа: *DSPR-38 (DSPR-1)*.
## Студент: *Светлаков Сергей*.
## Дата начала работы: *24.06.2021*.
## Дата сдачи проекта: *01.06.2021*.

### ***Цель работы***: создать и обучить модель на основе классических методов машинного обучения и нейронных сетей (MLP - многослойный перцептрон, RNN - рекурентная нейронная сеть, CNN - сверточная нейронная сеть) для решения задачи регрессии по предсказанию цен на автомибили по объявлениям с сайта auto.ru. В качестве изображений выступают фото машин. В качестве текста - описания объявлений. В качестве табличных данных - параметры автомобилей. Парсинг данных не требуется. Метрика: MAPE - средняя относительная погрешность. Данная работа выполнялась на основе имеющегося Base-Line решения на платформе SkillFactory.

***
***Таким текстом отображаюстя промежуточные выводы для финальной версии программы и результатов на Kaggle.***
***

***
<span style='color:Red'> ***Таким текстом отображаются промежуточные выводы, соответствующие предыдущим версия программы. Также так отображаются выводы "между строк" автора работы.*** </span>
***

***
***Скачиваем библиотеки, которые понадобятся для дальнейшей работы с изображениями и текстом в NLP и CV.***
***

In [ ]:
#Для CV
!pip install albumentations -q
!pip install -q efficientnet
#Для NLP
!pip install pymystem3
!pip install nltk

In [ ]:
#Импорт библиотек
#Данные
import numpy as np
import pandas as pd
#Визуализация
import matplotlib.pyplot as plt
import seaborn as sns
#Для тестов
from itertools import combinations
from scipy.stats import ttest_ind
#ML
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
#DL - MLP+
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.optimizers as O
from tensorflow.keras.models import Model as M, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
#DL - NLP
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from pymystem3 import Mystem
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
#DL - CV
import efficientnet.tfkeras as efn
import albumentations as alb
#Прочее
import random
import os
from string import punctuation
import re
import os
import sys
import PIL
import cv2

***
***Настраиваем randomize для воспроизведения результатов.***
***

***
<span style='color:Red'> ***К сожалению, чтобы я не делал, заставить notebook из раза в раз выдавать на Neural Network (в дальнейшем NN) один и тот же результат у меня не получилось. Он все равно выдает разное. См. подробнее в пункте 6.*** </span>
***

In [ ]:
#Настройки
sns.set(style="darkgrid", font_scale=1.0)
sns.set_palette("pastel")
#Для воспроизведения результатов
random_state = 42
tf.random.set_seed(random_state)
np.random.seed(random_state)
random.seed(random_state)
os.environ['PYTHONHASHSEED']=str(random_state)
#Для NLP
snowball = SnowballStemmer(language="russian")
mystem = Mystem()
punctuation+='•«»–'
delete_english = re.compile("[а-яА-Я]+")

# 0.Функции

***
***В данном блоке приведены функции, которые я написал специально для обработки данных - EDA и Feature engineering. Они позволяют мне повторять один и тот же процесс n-ое число раз, сокращая количество программного кода.***
***

***
<span style='color:Red'> ***Можно заметить, что в качестве целевого признака используется не заданный price, а будущий log_price. Подробнее в пункте 3.1.*** </span>
***

In [ ]:
def Is_drop(df,col,return_count=False):
    '''
    Функция для определения выброс прецендент или нет (или выводы количества выбросов).
    Вход:
    * df - DataFrame;
    * col - столбец в DataFrame, по которому считаются выбросы;
    * return_count - возвращать количество?.
    Выход:
    * Количество выбросов.
    '''
    #Квантили
    q25 = df[col].quantile(0.25)
    q75 = df[col].quantile(0.75)
    #Межквантильный размах
    IQR = q75 - q25
    #Выброс ли?
    df_sub = ~df[col].between(q25 - 1.5*IQR, q75 + 1.5*IQR)
    if return_count:
        return df_sub.sum()
    else:
        return df_sub

def Is_need_transform(df,col,func):
    '''
    Функция для вывода всей информации о необходимости преобразовании признака.
    Вход:
    * df - DataFrame;
    * col - столбец в DataFrame, по которому считается;
    * func - функция преобразования.
    Выход:
    * None.
    '''
    #Место для Hist-Plot и его настройки
    fig, axes = plt.subplots(1,2,figsize = (12,5))
    #Название
    axes[0].set_title('До преобразования')
    axes[1].set_title('После преобразования')
    #Графики
    #Изначальный
    df_sub = df.copy()
    drop_init = Is_drop(df_sub,col,True)
    sns.histplot(x=col, data=df_sub, ax=axes[0], element='bars', bins=25)
    #Конечный
    df_sub[col] = df_sub[col].apply(func)
    drop_end = Is_drop(df_sub,col,True)
    sns.histplot(x=col, data=df_sub, ax=axes[1], element='bars', bins=25)
    #Вывод информации
    print('Признак {} преобразован. Количество выбросов до: {}; после {}; сокращено: {}.'.\
         format(col,drop_init,drop_end,drop_init-drop_end))
    #Описание над графиком
    fig.suptitle('Hist-Plot for ' + col)
    pass

def get_info_feature(data,x,show_lost_uniq=True,show_uniq=False):
    '''
    Получение информации о признаке.
    Вход:
    * data - DataFrame;
    * x - признак;
    * show_lost_uniq - показать ли потерянные признаки в выборках?;
    * show_uniq - показать ли уникальные признаки?.
    Выход:
    * None.
    '''
    #Разделение выборок
    data_train = data.query('Kaggle==0')
    data_test  = data.query('Kaggle==1')
    #Количество уникальных значений
    nu_trn = data_train[x].nunique()
    nu_tst = data_test[x].nunique()
    #Количество значений
    ln_trn = len(data_train)
    ln_tst = len(data_test)
    #Количество пропусков
    na_trn = data_train[x].isna().sum()
    na_tst = data_test[x].isna().sum()
    #Уникальные значения
    uniq_trn = list(data_train[x].unique())
    uniq_tst = list(data_test[x].unique())
    #Вывод
    print('Количество уникальных  значений train: {} / {} - {}%.'.format(nu_trn,ln_trn,np.round(nu_trn/ln_trn*100,2)))
    print('Количество уникальных  значений test:  {} / {} - {}%.'.format(nu_tst,ln_tst,np.round(nu_tst/ln_tst*100,2)))
    print('Количество пропущенных значений train: {} / {} - {}%.'.format(na_trn,ln_trn,np.round(na_trn/ln_trn*100,2)))
    print('Количество пропущенных значений test:  {} / {} - {}%.'.format(na_tst,ln_tst,np.round(na_tst/ln_tst*100,2)))
    if show_lost_uniq:
        lost_trn = []
        lost_tst = []
        for i in uniq_trn:
            if i not in uniq_tst:
                lost_trn += [i]
        for i in uniq_tst:
            if i not in uniq_trn:
                lost_tst += [i]
        print('Отсутствующие значения в train: ',lost_tst)
        print('Отсутствующие значения в test:  ',lost_trn)
    if show_uniq:
        print('Уникальные значения в train: ',uniq_trn)
        print('Уникальные значения в train: ',uniq_tst)
    pass

def get_corr(data,x,y='log_price'):
    '''
    Получение корреляции параметров с целевым.
    Вход:
    * data - DataFrame;
    * x - список из признаков;
    * y - целевой признак.
    Выход:
    * Таблица корреляции.
    '''
    #Обучающая выборка
    data_sub = data.query('Kaggle==0')
    #Если передан один признак
    if type(x) != list:
        x = [x]
    #Таблица корреляция в %
    table_corr = data_sub[x+[y]].corr() * 100
    return table_corr.round(2)

def get_order(data,x,y='log_price',st='mean'):
    '''
    Функция для получения порядка значений категориального признака по значению целевого признака (используется для box-plot).
    Вход:
    * data - DataFrame;
    * x - список из признаков;
    * y - целевой признак;
    * st - агрегирующая функция для вычисления порядка (mean, median и т.д.).
    Выход:
    * Порядок значений.
    '''
    return data.query('Kaggle==0').groupby(x)[y].agg(st).sort_values().index

#Функция для определения статически значимых
def Is_stat_dif(x,y,df,alpha=0.05):
    '''
    Поиск статически значимых параметров.
    Вход:
    * x - название столбца в DataFrame, по которому группируются данные;
    * y - название столбца в DataFrame, по которому считается доверительный интервал;
    * df - DataFrame;
    * alpha=0.05 - уровень значимости.
    Выход:
    * Список из элементов [статически не значим, статически значим].
    '''
    #Список групп
    ind = df.loc[:, x].value_counts().index
    #Создание различных комбинаций из списка по 2
    combo = list(combinations(ind, 2))
    #Поиск
    for comb in combo:
        #Определение p-уровня значимости
        p = ttest_ind(df.loc[df.loc[:, x] == comb[0], y],
                     df.loc[df.loc[:, x] == comb[1], y]).pvalue
        #Проверка (знаменатель необходим для учета поправки Бонферрони)
        if p <= alpha / len(combo):
            print('Статистически значим: {}'.format(x))
            return [np.nan, x]
    else:
        print('Статистически не значим: {}'.format(x))
        return [x, np.nan]
    pass

***
<span style='color:Red'> ***Здесь и далее корреляция выражена в %, а не в [-1,+1].*** </span>
***

***
***Функции для визуализации.***
***

In [ ]:
def boxplot(data,x,y='log_price',size=(20,8),hue=None,st='mean',showmeans=True):
    #Box-Plot
    fig,axes = plt.subplots(1,1,figsize=size)
    sns.boxplot(x=x,y=y,data=data.query('Kaggle==0'),hue=hue,
                order=get_order(df,x,st=st),
                showmeans=showmeans,meanprops={"marker":"o","markerfacecolor":"white","markeredgecolor":"black","markersize":"5"})
    pass

def scatterplot(data,x,y='log_price',size=(20,8),hue=None):
    #Scatter-Plot
    fig,axes = plt.subplots(1,1,figsize=size)
    sns.scatterplot(x=x,y=y,data=data.query('Kaggle==0'),hue=hue)
    pass

def histplot(data,x,size=(20,8),hue=None):
    #Hist-Plot
    fig,axes = plt.subplots(1,1,figsize=size)
    sns.histplot(x=x,data=data.query('Kaggle==0'),hue=hue)
    pass

def heatmap_corr(df,cols=None,size=(20,20)):
    '''
    Построение тепловой карты по матрице корреляций.
    Вход:
    * df - DataFrame;
    * cols - столбцы в DataFrame, по которым считаются корреляции;
    * size - размер графика.
    Выход:
    * None.
    '''
    #Plot
    fig, ax = plt.subplots(figsize = size)
    #Title
    ax.set_title('HeatMap for Correlation')
    if cols==None:
        cols=df.columns
    #Table of corr
    table_corr = df[cols].corr().round(2)
    #Plot-Seaborn
    sns.heatmap(table_corr, vmin=-1, vmax=1, cmap="YlGnBu",annot=True)
    #Show
    plt.yticks(rotation=0) 
    plt.show()
    pass

def scatterhistplot(data,x,y='log_price',size=(20,8),hue=None):
    '''
    Построение двух графиков - scatter & hist plot'ы.
    Вход:
    * data - DataFrame;
    * x - признак в DataFrame по оси абсцисс scatterplot;
    * y - целевой признак по оси ординат scatterplot;
    * size - размер графика;
    * hue - дополнительное разделение по группам.
    Выход:
    * None.
    '''
    #Scatter-Plot & Гистограмма
    fig,axes = plt.subplots(1,2,figsize=size)
    sns.scatterplot(x=x,y=y,data=data.query('Kaggle==0'),ax=axes[0],hue=hue)
    sns.histplot(x=x,data=data.query('Kaggle==0'),ax=axes[1],hue=hue)
    pass

***
***Еще функции для EDA: первая для обработки номинативных признаков, вторая - числовых.***
***

In [ ]:
def info_nom(df,x,y='log_price',size=(20,8),st='mean',show_boxplot=True):
    #Тип признака
    print('Тип признака: ',df[x].dtypes)
    #Количество уникальных значений
    get_info_feature(df,x)
    #Box-Plot
    if show_boxplot:
        boxplot(df,x,y=y,size=size,st=st)
    #Распределение
    print('Распределение значений в train:')
    print(df.query('Kaggle==0')[x].value_counts())
    pass

def info_num(df,x,y='log_price',size=(20,8),func=None,show_plot=True):
    #Количество уникальных значений
    get_info_feature(df,x,False)
    #Корреляция с целевым признаком
    print('Корреляция с целевым признаком:')
    print(get_corr(df,x))
    #Scatter-Plot & Гистограмма
    if show_plot:
        scatterhistplot(df,x,y=y,size=size)
    #Нужно ли трасформировать
    if func:
        Is_need_transform(df,x,func)
    pass


***
***Функции для ML-решений.***
***

In [ ]:
def mape(y_true, y_pred, show=False):
    '''
    Метка качества для ML моделей: mean_absolute_percentage_error - mape.
    Вход:
    * y_true - массив истинных значений;
    * y_pred - массив предсказанных значений;
    * show - вывести значение?
    Выход:
    * Метрика MAPE.
    '''
    met = np.mean(np.abs((y_pred-y_true)/y_true)) * 100
    #Вывод информации
    if show:
        print('mape={:.2f}%'.format(met))
    return met

def predict_to_Kaggle(model,X_test,name='submission'):
    '''
    Сохранение результатов на Kaggle.
    Вход:
    * model - обученная модель;
    * X_test - матрица прецендентов для предсказания;
    * name - название файла.
    Выход:
    * None.
    '''
    #Предсказание на test
    Y_pred_test = model.predict(X_test)
    #Пост-обработка - удаление логорифмирования целевого признака
    Y_pred_test = np.exp(Y_pred_test)
    #Создание DF для вывода
    sample_submission = pd.read_csv(PATH + 'sample_submission.csv')
    sample_submission['price'] = Y_pred_test
    #Вывод в файл
    sample_submission.to_csv(name+'.csv', index=False)
    pass

# 1.Чтение и осмотр данных

***
<span style='color:Red'> ***Notebook изначально писался на собственном ПК. Как подключить его к соревнованию на Kaggle - я не понял, поэтому я скачал данные input и поменял путь PATH.*** </span>
***

In [ ]:
#Путь
PATH = '../input/sf-data-module-8/'
#Чтение
train = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'test.csv')
#Метка
train['Kaggle'] = 0
test['Kaggle'] = 1
test['price'] = np.nan
#Объединение
df = pd.concat([train,test],axis=0)

In [ ]:
#Осмотр
df.head(3)

***
***Много необработанных признаков, содержащих не числовые значения.***
***

In [ ]:
#Пропуски
df_sub = df.groupby('Kaggle').agg(lambda x: x.isna().sum()).sum()
#Распределение по выборкам
df[list(df_sub[df_sub > 0].index)+['Kaggle']].groupby('Kaggle').agg(lambda x: x.isna().sum())

***
***Присутствуют признаки (Владельцы, Владение) с пропущенными значениями.***
***

# 2.Наивная модель

***
***Чтобы от чего-то отталкиваться при построении ML-моделей построим простую наивную модель и зафиксируем метрику. Будем считать, что хуже этой метрики получить нельзя.***
***

In [ ]:
#Разделение выборки на обучающую и валидационную
trn, val = train_test_split(train,test_size=0.15,shuffle=True,random_state=random_state)

In [ ]:
#Наивная модель
predicts = []
for index, row in pd.DataFrame(val[['model_info', 'productionDate']]).iterrows():
    query = f"model_info == '{row[0]}' and productionDate == {row[1]}"
    predicts.append(trn.query(query)['price'].median())
#Если не нашлись совпадение, то медианным значением
predicts = pd.DataFrame(predicts)
predicts = predicts.fillna(predicts.median())
#Вывод
print(f"Точность наивной модели по MAPE: {(mape(val['price'], predicts.values[:, 0])):0.2f}%.")

***
***Точность 19.88%. Для улучшения произведем обработку имеющихся признаков и создадим на основе их новые.***
***

# 3.EDA & Feature engineering

***
***Рассмотрим каждый признак по отдельности. После обработки признаков будем записывать их в различные группы в соответствии с их типом: числовые, номинативные (бинарные, категориальные).***
***

***
<span style='color:Red'> ***Достаточного количества ординальных признаков тут не получилось, поэтому я им не добавлял отдельную группу. Записывал в категориальные (например, n_owners).*** </span>
***

In [ ]:
col_bin = [] #Бинарные
col_cat = [] #Категориальные
col_num = [] #Числовые
col_drp = [] #Для удаления

## 3.1) price

***
***Сразу обработаем целевой признак.***
***

In [ ]:
#Нужно ли преобразовывать?
Is_need_transform(df.query('Kaggle==0'),'price',lambda x: np.log(x))
#Преобразование
df['log_price'] = df['price'].apply(np.log)

***
***Видно, что исходный признак price распределен экспоненциально. Это может привести к затруднениям при обучении NN. Приведем к нормальному виду с помощью логорифмирования. Будем считать, что полученный признак log_price является целевым. При обработке будущих предсказаний необходимо дополнительно возводить в exp.***
***

***
<span style='color:Red'> ***Были попытки работать с price напрямую. Алгоритм CatBoost выдавал на 1.5-2.5% результат хуже. А NN стабильно выдавала метрику MAPE 100%.*** </span>
***

In [ ]:
col_drp += ['price']

## 3.2) bodyType

***
***Здесь и далее я не буду заострять много внимания на таких шагах, как переименование признаков (делал для собственного удобства и читаемости) и преобразование признаков (логорифмирование или извлечение корня, чтобы сократить число выбросов и привести к нормальному виду распределения величин).***
***

In [ ]:
#Переименование
df.rename(columns={'bodyType':'type_body'},inplace=True)

In [ ]:
#Информация по признаку
info_nom(df,'type_body',size=(40,8))

***
***Видно не вооруженным глазом, что число признак статистически значим. Но количество уникальных значений (групп) чрезмерно большое. Также имеются группы, где количество прецендентов меньше 10-30. Преобразуем данный признак, объединив несколько категорий в одну.***
***

In [ ]:
#Переименование параметров
df['type_body'].replace({
    'седан 2 дв.':'седан',
    'пикап двойная кабина':'лимузин',
    'внедорожник открытый':'купе',
    'внедорожник 3 дв.':'седан',
    'компактвэн':'внедорожник открытый',
    #Дополнительная группа - ДГ
    'купе':'ДГ',
    'открытый купе':'ДГ',
    'лифтбек':'ДГ',
    'кабриолет':'ДГ',
    'родстер':'ДГ',
    'внедорожник 5 дв.':'ДГ'
}, inplace=True)

***
***Категории объединялись по следующему принципу: категории с количеством прецендентов меньше 10 объединялись с ближайшими по среднему значению; часть категорий были объединены в одну большую группу ДГ, так как все они похожи между собой - примерно одно значение среднего и медианы (но немного разные IQR).***
***

In [ ]:
boxplot(df,'type_body')

***
***Количество категорий сократилось до 10. Видно, что самые дорогие машины - это лимузины и экзотика на территории РФ - пикап. Самыми дешевыми являются повсеместные седаны и хетчбеки. Диапазон средних цен занимают внедорожники, минивэны.***
***

In [ ]:
col_cat += ['type_body']

## 3.3) brand

In [ ]:
#Информация по признаку
info_nom(df,'brand')

***
***В отличие от проекта 6, количество брендов автомобилей тут сокращено до трех. Все три являются престижными, поэтому выделять их в отдельный признак не стоит. Распределение целевого признака каждой группы статистически значимо и имеет нормальный характер (небольшое смещение вправо - в область низких цен, хвост длинный в область высоких цен).***
***

In [ ]:
col_cat += ['brand']

## 3.4) color

In [ ]:
#Информация по признаку
info_nom(df,'color')

***
***Уникальных значений также очень много. Сократим, объединив категории с малым количеством прецендентов.***
***

In [ ]:
#Переименование параметров
df['color'].replace({
    'жёлтый':'желтый',
    'розовый':'желтый',
    'оранжевый':'желтый'
}, inplace=True)

In [ ]:
boxplot(df,'color')

***
***Выделим стандартные цвета в отдельный признак.***
***

***
<span style='color:Red'> ***Возможно стоило тут создать признак на основе частоты встречаемости цвета.*** </span>
***

In [ ]:
#Стандартные цвета
df['is_standart_color'] = df['color'].apply(lambda x: x in ['серый','белый','черный']).astype(int)

In [ ]:
boxplot(df,'is_standart_color')

***
***Скорее всего признак не является статистически значимым. Дальнейший анализ покажет это.***
***

In [ ]:
col_cat += ['color']
col_bin += ['is_standart_color']

## 3.5) description

***
***Признак description содержит текстовое описание объявления по автомобилю. Сам текст пригодится для рекурентных NN (в дальнейшем RNN). Но попробуем заранее для классических ML моделей извлечь полезные признаки.***
***

***
***До обработки текста извлечем его длину, число слов, среднюю длину слова, количество отдельных знаков препинания и т.д.***
***

In [ ]:
#Длина описания
df['len_description'] = df['description'].apply(len)
#Количество слов
df['n_word'] = df['description'].apply(lambda x: len(x.split(' ')))

In [ ]:
#Число знаков препинания, пробелов и т.д.
df['description_n_point'] = df['description'].apply(lambda x: x.count('.'))
df['description_n_comma'] = df['description'].apply(lambda x: x.count(','))
df['description_n_excla'] = df['description'].apply(lambda x: x.count('!'))
df['description_n_quest'] = df['description'].apply(lambda x: x.count('?'))
df['description_n_colon'] = df['description'].apply(lambda x: x.count(':'))
df['description_n_semic'] = df['description'].apply(lambda x: x.count(';'))
df['description_n_multi'] = df['description'].apply(lambda x: x.count('*'))
df['description_n_divis'] = df['description'].apply(lambda x: x.count('/'))
df['description_n_perce'] = df['description'].apply(lambda x: x.count('%'))
df['description_n_space'] = df['description'].apply(lambda x: x.count(' '))
df['description_n_minus'] = df['description'].apply(lambda x: x.count('-'))
df['description_n_plus']  = df['description'].apply(lambda x: x.count('+'))
#Все знаки препинания
df['description_n_all'] = df['description_n_point'] + df['description_n_comma'] + df['description_n_excla'] +\
                          df['description_n_quest'] + df['description_n_colon'] + df['description_n_semic'] +\
                          df['description_n_multi'] + df['description_n_divis'] + df['description_n_perce'] +\
                          df['description_n_space'] + df['description_n_minus'] + df['description_n_plus']

In [ ]:
#Предположительно средняя длина предложения
df['len_sentence'] = df['len_description'] / (1 + df['description_n_point'])
#Число цифр
df['len_digit'] = df['description'].apply(lambda x: sum(i.isdigit() for i in x))
#Средняя длина слов
df['len_word'] = df['description'].apply(lambda x: np.mean([len(i) for i in x.split(' ')]))

***
***Далее посмотрим на каждый признак и обработаем их.***
***

***
<span style='color:Red'> ***Осмотр каждого признака сокращен для чистоты кода, приведена лишь конечная обработка тех признаков, которые действительно нужно обрабатывать.*** </span>
***

In [ ]:
#Нужно ли преобразовывать?
Is_need_transform(df.query('Kaggle==0'),'len_word',lambda x: np.log(1+x))
#Информация по признаку
info_num(df,'len_word',size=(12,6))

***
***Признак len_word - средняя длина слова в отличие от всех остальных имеет нормальное распределение. Здесь и далее любая корреляция с целевым признаком выше 0.15 по модулю будет считаться полезной.***
***

In [ ]:
for name in ['len_description','description_n_point','description_n_comma',
             'description_n_excla','description_n_colon',
             'description_n_space','description_n_minus',
             'description_n_all','len_sentence','len_digit','n_word']:
    #Нужно ли преобразовывать?
    Is_need_transform(df.query('Kaggle==0'),name,lambda x: np.log(1+x))
    #Преобразование
    df[name] = df[name].apply(lambda x: np.log(1+x))
    #Информация по признаку
    info_num(df,name,show_plot=False)

***
***Признаки приведенные выше имеют исходное распределение близкое к экспоненциальному. Преобразуем признак с помощью логорифмирования. Стоит заметить ряд важных зависимостей: чем больше описание (количество слов и количество символов пунктуации соответственно), тем выше цена - корреляция ~+0.2.***
***

In [ ]:
col_drp += ['description']
col_num += ['len_description','description_n_point','description_n_comma',
             'description_n_excla','description_n_quest','description_n_colon',
             'description_n_semic','description_n_multi','description_n_divis',
             'description_n_perce','description_n_space','description_n_minus',
             'description_n_plus','description_n_all','len_sentence','len_digit','len_word','n_word']

***
<span style='color:Red'> ***Дальнейшая обработка признака приведена в пункте 4.2.*** </span>
***

## 3.6) engineDisplacement

In [ ]:
#Информация по признаку
info_nom(df,'engineDisplacement',show_boxplot=False)

***
***Признак имеет тип string и нуждается в преобразовании. Можно заметить, что признак имеет вид: [[число] [пробел] [LTR]]. Можно извлечь число, являющееся по смыслу объемом двигателя автомобиля. Также заменим неизвестное значение undefined значением близким, имеющим похожее распределение на box-plot.***
***

In [ ]:
#Замена
df['engineDisplacement'].replace({'undefined LTR':'6.0 LTR'}, inplace=True)

In [ ]:
#Извлечение
df['volume_engine'] = df['engineDisplacement'].apply(lambda x: x.split(' ')[0]).astype(float)

In [ ]:
info_num(df,'volume_engine')

***
***Видна закономерность, что с ростом объема двигателя растет его цена. Скорее всего это связано с тем, что рост объема двигателя сопровождается переходом от легкового автомобиля к грузовым (вэны и пикапы) или спорткарам.***
***

In [ ]:
col_drp += ['engineDisplacement']
col_num += ['volume_engine']

***
<span style='color:Red'> ***Признак будет прологорифмирован позже в пункте 4.1.*** </span>
***

## 3.7) enginePower

In [ ]:
#Информация по признаку
info_nom(df,'enginePower',show_boxplot=False)

***
***Аналогично предыдущему признаку видно, что признак представлен в виде string и имеет вид: [[N12] [пробел] [число]]. Число имеет физический смысл - мощность двигателя.***
***

In [ ]:
#Извлечение
df['power_engine'] = df['enginePower'].apply(lambda x: x.split(' ')[0]).astype(float)

In [ ]:
#Информация по признаку
info_num(df,'power_engine')

***
***Определена хорошая положительная корреляция между целевым признаком и power_engine ~+0.5. С ростом мощности двигателя растет и цена автомобиля, что логично. Ведь мощные двигатели свойственны либо споркарам, либо грузовым автомобилям (внедорожники, вэны, пикапы).***
***

***
***В принципе на этом можно было остановиться, но важно извлечь из этих двух признаков третий - экономичность двигателя. Все хотят мощный двигатель, но, как правило, увеличение мощности двигателя приводит к увеличению его объема, что, в свою очередь, увеличивает траты автомобилиста на топливо. Поэтому можно утверждать, что автомобилист хочет более мощный двигатель, но с минимальным объемом. Составим такой признак.***
***

In [ ]:
#Мощность на объем
df['power_per_volume'] = df['power_engine'] / df['volume_engine']

In [ ]:
info_num(df,'power_per_volume')

***
***Получена хорошая положительная корреляция ~+0.5. Предыдущий вывод подтвержден - чем больше данный показатель, тем круче автомобиль, тем выше цена. Важно, чтобы данный признак не был сильно скоррелирован с двумя предыдущими.***
***

In [ ]:
col_drp += ['enginePower']
col_num += ['power_engine','power_per_volume']

***
<span style='color:Red'> ***Признак power_engine будет прологорифмирован позже в пункте 4.1.*** </span>
***

## 3.8) fuelType

In [ ]:
#Переименование
df.rename(columns={'fuelType':'type_fuel'},inplace=True)

In [ ]:
#Информация по признаку
info_nom(df,'type_fuel')

***
***Две категории с маленьким числом прецендентов: электро и гибрид, статистически значимо отличаются от остальных. Объединим их в одну, чтобы уменьшить общее число категорий.***
***

In [ ]:
#point-plot
sns.catplot(data=df.query('Kaggle==0'),x='type_fuel',y='log_price',kind='point',aspect=3,hue='brand')

***
***Лишний раз убеждаемся, что объединение не было лишним. Алгоритм на подобие дерева решений сможет сначала разделить преценденты по бренду, а потом работать с каждой категорий по отдельности: электро и гибрид. Также видно, что для каждой категории type_fuel сохраняется закономерность MERCEDES>BMW>AUDI.***
***

In [ ]:
#Замена
df['type_fuel'].replace({'электро':'гибрид'}, inplace=True)

In [ ]:
col_cat += ['type_fuel']

## 3.9) mileage

In [ ]:
info_num(df,'mileage')

***
***Признак имеет нормальное распределение, сильно смещенное вправо с длинным хвостом, содержащие высокие значения. Необходимо преобразование. Также видна сильная отрицательная корреляция ~-0.7: с ростом пробега падает цена.***
***

In [ ]:
#Нужно ли преобразовывать?
Is_need_transform(df.query('Kaggle==0'),'mileage',lambda x: x**0.5)
#Преобразование
df['sqr_mileage'] = df['mileage'].apply(lambda x: x**0.5)

***
<span style='color:Red'> ***Иногда вместо логорифмирования применяется функция квадратного корня. Выбирается оптимальная.*** </span>
***

In [ ]:
info_num(df,'sqr_mileage')

***
***После преобразования корреляция становится чуточку сильнее.***
***

In [ ]:
col_drp += ['mileage']
col_num += ['sqr_mileage']

## 3.10) model_info

In [ ]:
#Переименование
df.rename(columns={'model_info':'model'},inplace=True)

In [ ]:
#Информация по признаку
info_nom(df,'model')

***
<span style='color:Red'> ***Самый интересный и важный признак, на мой взгляд. Не обращаем внимания на ось X, важна лишь ось Y. Видно, что группы автомобилей, разделенные по признаку model, сильно различаются между собой. Но количество уникальных значений катастрофически высокое. Есть три пути решения проблемы: оставить все как есть, сократить количество категорий, создать признак на основе целевого и данного признака - наивный целевой признак. Первый вариант - не наш вариант, так как при тестах модель показала худшую точность. Выбирая между вторым и третьим вариантами я отдал предпочтение последнему, так как полученный признак будет сильно скоррелирован с целевым. Это конечно может привести к переобучению, но тесты показали, что так точность выше по метрике MAPE на 0.5% в сравнении со вторым вариантом на Kaggle.*** </span>
***

***
***Составим новый признак на основе имеющегося, создав наивную цену на основе комбинации трех признаков: brand, model, modelDate.***
***

In [ ]:
#Группировка по brand, model, modelDate
df_sub_1 = df.groupby(['brand','model','modelDate'])['log_price'].agg('mean')
df_sub_2 = df.groupby(['brand','model'])['log_price'].agg('mean')
df_sub_3 = df.groupby(['brand'])['log_price'].agg('mean')
#Замена значений
def get_naive_price(x):
    if np.isnan(df_sub_1[tuple(x.values)]):
        if np.isnan(df_sub_2[tuple(x.values)[:-1]]):
            return df_sub_3[tuple(x.values)[0]]
        else:
            return df_sub_2[tuple(x.values)[:-1]]
    else:
        return df_sub_1[tuple(x.values)]
#Создание нового признака - наивная цена по группам brand, model, modelDate
df['log_price_naive'] = df[['brand','model','modelDate']].apply(get_naive_price, axis=1)

In [ ]:
info_num(df,'log_price_naive')

***
***Что и ожидалось - признак сильно скоррелирован. Распределение около-нормальное, преобразование не требуется.***
***

In [ ]:
col_drp += ['model']
col_num += ['log_price_naive']

## 3.11) name

In [ ]:
#Осмотр
df['name'].head(5)

***
***На первый взгляд данный признак уже содержит имеющиеся: мощность и объем двигателя. Но помимо этого, он может содержать дополнительные приписки типов двигателя: AT, WD и т.д. Извлечем их. Список данных имен выбирается на основе общего числа встречаемости по DataFrame'у: больше 15 упоминаний.***
***

In [ ]:
#Поиск значений и составление на основе их признаков
for name in ['AT','WD','AMT','xDrive','CVT','AMG','Long',
             'CDI','MT','BlueTEC','BlueEFFICIENCY','Competition',
             'S-tronic','экстра']:
    df['name_'+name] = df['name'].apply(lambda x: int(name in x))

In [ ]:
col_drp += ['name']
for name in ['AT','WD','AMT','xDrive','CVT','AMG','Long',
             'CDI','MT','BlueTEC','BlueEFFICIENCY','Competition',
             'S-tronic','экстра']:
    col_bin += ['name_'+name]

***
***Рассмотрим некоторые из них, чтобы убедиться, что извлечение данных признаков было статистически значимо в будущем.***
***

In [ ]:
for name in ['WD','xDrive','CVT','AMG','BlueTEC','Competition','экстра']:
    boxplot(df,'name_'+name,size=(20,4))

***
***Действительно, данные признаки позволяют отделить от общей генеральной совокупности подвыборки с сильно низкой/высокой ценой.***
***

## 3.12) numberOfDoors

In [ ]:
#Переименование
df.rename(columns={'numberOfDoors':'n_doors'},inplace=True)

In [ ]:
#Информация по признаку
info_nom(df,'n_doors')

***
***Признак является категориальным. Количество дверей скорее всего сильно скоррелирово с типом кузова: type_body.***
***

In [ ]:
col_cat += ['n_doors']

## 3.13) sell_id

***
***Данный признак не рассматривается напрямую, а лишь является ссылкой на фотографию автомобиля. Пригодится для сверточных нейронных сетей (в дальнейшем CNN).***
***

In [ ]:
col_drp += ['sell_id']

## 3.14) vehicleConfiguration

In [ ]:
#Осмотр
df[['vehicleConfiguration','volume_engine','type_body','vehicleTransmission']].head(10)

***
***Видно, что данный признак является комбинацией трех других. Удалим его.***
***

In [ ]:
col_drp += ['vehicleConfiguration']

## 3.15) vehicleTransmission

In [ ]:
#Переименование
df.rename(columns={'vehicleTransmission':'type_transmission'},inplace=True)

In [ ]:
#Информация по признаку
info_nom(df,'type_transmission')

***
***Самая большая категория - автоматическая. Имеет при этом самый высокий разброс. Роботизированная на первый взгляд не статистически значимо отличается от автоматической, но при этом две другие категории имеют сильно низкую цену относительно первых двух. Разберемся поподробнее почему автоматическая так похожа на роботизированную, а вариантор на механическую.***
***

In [ ]:
#box-plot
sns.catplot(data=df.query('Kaggle==0'),x='type_transmission',y='log_price',kind='box',aspect=2,hue='brand',col='type_fuel')

***
<span style='color:Red'> ***Разделив генеральную совокупность по трем признакам: type_fuel, type_transmission и brand, видно, что при таком разделении все полученные подвыборки статистически значимо различаются. Стоит отметить, что гибрид содержит лишь автоматическую коробку передач, а BMW не имеет вариатор коробку передач. Также не существует группы BMW роботизированная на дизеле. Для бензина и дизеля можно утверждать, что в среднем цена на роботизированную коробку передач выше, чем на автоматическую по имеющемся брендам автомобилей.*** </span>
***

***
<span style='color:Red'> ***На самом деле можно сделать еще больше выводов по данным графикам и скорее всего на основе этих выводов сделать мета-признак, объединяющий все перечисленные раньше категориальные, позволяющий однозначно и точно разделить генеральную совокупность на статистически значимые подвыборки.*** </span>
***

In [ ]:
col_cat += ['type_transmission']

## 3.16) Владельцы

In [ ]:
#Переименование
df.rename(columns={'Владельцы':'n_owners'},inplace=True)

In [ ]:
#Информация по признаку
info_nom(df,'n_owners')

***
***Имеется один пропуск. С ростом числа владельцев падает цена. Заменим пропуск самым худшим вариантом - 3 или более владельца, ориентируясь на то, что скорее всего данный датасет был собран с сайта auto.ru, а если там не было указано число владельцев, то скорее всего продавец хотел скрыть это (а скрывать имеет смысл то, что может снизить цену).***
***

In [ ]:
#Заполнение пропуска
df['n_owners'].fillna('3 или более',inplace=True)

In [ ]:
#Замена
df['n_owners'].replace({'1\xa0владелец':'1','2\xa0владельца':'2','3 или более':'3+'},inplace=True)

In [ ]:
col_cat += ['n_owners']

## 3.17) ПТС

In [ ]:
#Переименование
df.rename(columns={'ПТС':'pts'},inplace=True)

In [ ]:
#Информация по признаку
info_nom(df,'pts')

***
***Также логично, что дубликат ПТС снижает цену: либо это копия потерянного, либо большое число владельцев, либо продавец хотел что-то скрыть.***
***

In [ ]:
col_bin += ['pts']

## 3.18) Привод

In [ ]:
#Переименование
df.rename(columns={'Привод':'drive'},inplace=True)

In [ ]:
#Информация по признаку
info_nom(df,'drive')

***
***Группы разбиты не равномерно, но достаточно для того, чтобы с ними работать. Передний привод самый дешевый, а полный самый дорогой.***
***

In [ ]:
col_cat += ['drive']

## 3.19) Руль

In [ ]:
#Переименование
df.rename(columns={'Руль':'type_steering'},inplace=True)

In [ ]:
#Информация по признаку
info_nom(df,'type_steering')

***
***Так как это объявления с сайта auto.tu, ориентированного на российский рынок, то наличие правого руля - это лишняя экзотика, которая вызывает трудности при эксплуатации автомобиля на дороге, что, в свою очередь, снижает его цену.***
***

In [ ]:
col_drp += ['type_steering']

## 3.20) modelDate

In [ ]:
#Информация по признаку
info_num(df,'modelDate')

***
***Есть положительная корреляция, что с чем моложе модель автомобиля, тем выше его цена. Признак имеет нормальное распределение, смешенное влево, исправим это.***
***

In [ ]:
#Замена
df['new_model_date'] = df['modelDate'].apply(lambda x: np.log(2020-x+1))

In [ ]:
#Информация по признаку
info_num(df,'new_model_date')

***
***После преобразования признак стал больше похож на нормальный, при этом увеличилась (по модулю) корреляция с целевым признаком.***
***

In [ ]:
col_num += ['new_model_date']
col_drp += ['modelDate']

## 3.21) productionDate

In [ ]:
#Информация по признаку
info_num(df,'productionDate')

***
***Аналогично modelDate.***
***

In [ ]:
#Замена
df['new_prod_date'] = df['productionDate'].apply(lambda x: np.log(2020-x+1))

In [ ]:
#Информация по признаку
info_num(df,'new_prod_date')

***
***Корреляция сильнее, чем у new_model_date. Извлечем дополнительный признак - разницу между годом модели и годом ее выпуска (ведь модель автомобиля может быть выпущена несколько раз в разные года после запуска первой серии).***
***

In [ ]:
#Задержка между годом создания и годом выпуска
df['pause_year'] = df['productionDate'] - df['modelDate']

In [ ]:
#Информация по признаку
info_num(df,'pause_year')

***
***Получилось нормальное распределение (почти) со слабой корреляцией. Устраним хвост.***
***

In [ ]:
#Нужно ли преобразовывать?
Is_need_transform(df.query('Kaggle==0'),'pause_year',lambda x: np.log(1+x))
#Преобразование
df['log_pause_year'] = df['pause_year'].apply(lambda x: x**0.5)

In [ ]:
col_num += ['new_prod_date','log_pause_year']
col_drp += ['productionDate','pause_year']

## 3.22) Владение

In [ ]:
#Осмотр
df['Владение'].dropna().head(10)

***
***Признак содержит строковое представление возраста (срока эксплуатации) автомобиля. Обработаем сначала не NaN значения, а затем NaN.***
***

In [ ]:
def extract_age(x):
    if str(x) != 'nan':
        #Разделение на массив
        x = x.split(' ')
        #Извлечение
        if len(x) > 2:
            m = x[3]
            y = x[0]
            return int(y) * 12 + int(m)
        else:
            if x[1] in ['лет', 'года']:
                return int(x[0]) * 12
            else:
                return int(x[0])
    else:
        return np.nan
#Общее число месяцев проката
df['Age'] = df['Владение'].apply(extract_age)

In [ ]:
#Информация по признаку
info_num(df,'Age')

***
***Присутствует небольшая слабая корреляция: с ростом времени эксплуатации автомобиля падает его цена.***
***

***
***Признак наполовину пустой, выделим наличие пропуска у прецендента в отдельный признак.***
***

In [ ]:
#Было ли это пустое значение?
df['Is_NaN_Age'] = df['Age'].isna().astype(int)

In [ ]:
#Информация по признаку
info_nom(df,'Is_NaN_Age')

***
***Не статистически значим.***
***

***
***Далее попробуем обработать пропуск. Можно заполнить все медианным/средним значением. Можно не просто посчитать медианное/среднее для генеральной совокупности, а для отдельных выборок это сделать и присвоить полученные значения прецендентам с NaN в соответсвии с их категориями.***
***

***
***Но можно поступить по-другому. Предположим, что сейчас 2020 или 2021 год. Тогда если сделать гипотезу, что автомобиль с NaN был введен в эксплуатацию с момента его даты производства, то можно, таким образом, определить срок его владения.***
***

In [ ]:
#Текущий год
(df['productionDate'] + df['Age'] // 12).max()

***
<span style='color:Red'> ***Максимальный год в выборке все же действительно 2020, а не 2021.*** </span>
***

In [ ]:
#Заполнение пропусков
df.loc[df['Age'].isna(),'Age'] = (2020 - df.loc[df['Age'].isna(),'productionDate']) * 12

***
***Признак Age приведен в месяцах.***
***

In [ ]:
#Информация по признаку
info_num(df,'Age')

***
***По сравнению с не очищенным признаком корреляция с целевым признаком увеличилась в два раза. Преобразуем - сократим выбросы.***
***

In [ ]:
#Нужно ли преобразовывать?
Is_need_transform(df.query('Kaggle==0'),'Age',lambda x: x**0.5)
#Преобразование
df['sqr_age'] = df['Age'].apply(lambda x: x**0.5)

***
***Дополнительно выдвинем гипотезу, что все, кто делает дубликат ПТС, хотят скрыть что-то (снижает цену), либо число владельцев превысило допустимое число (снижает цену). Скомбинируем данные признаки и сделаем новый на основе худшего прогноза.***
***

In [ ]:
#Новое число владельцев с учетом pts
df['new_n_owners'] = df['n_owners'].replace({'1':1,'2':2,'3+':3})
df['new_n_owners'] = df[['pts','new_n_owners']].apply(lambda x: x[1] if x[0] == 'Оригинал' else x[1]+1, axis=1)

In [ ]:
#Информация по признаку
info_nom(df,'new_n_owners')

***
***Тенденция сохраняется - с ростом числа владельцев падает цена.***
***

***
***Определим средний срок владения автомобиля каждым владельцем.***
***

In [ ]:
#Средний срок владения одним владельцем
df['Age_per_owners'] = df['Age'] / df['n_owners'].replace({'1':1,'2':2,'3+':3})

In [ ]:
#Информация по признаку
info_num(df,'Age_per_owners')

***
***Распределение необходимо преобразовать, так как оно сильно смещено вправо.***
***

In [ ]:
#Нужно ли преобразовывать?
Is_need_transform(df.query('Kaggle==0'),'Age_per_owners',lambda x: x**0.5)
#Преобразование
df['sqr_age_per_owners'] = df['Age_per_owners'].apply(lambda x: x**0.5)

In [ ]:
#Информация по признаку
info_num(df,'sqr_age_per_owners')

***
***Есть небольшая отрицательная корреляция, что чем выше срок владения одним владельцем, тем ниже цена. Скорее всего это связано напрямую со сроком владения автомобиля.***
***

***
***Составим новый признак - средний срок пробега в месяц.***
***

In [ ]:
#Средний пробег машины в месяц
df['mileage_per_age'] = df['mileage'] / (df['Age'] + 1)

In [ ]:
#Информация по признаку
info_num(df,'mileage_per_age')

***
***Есть несколько автомобилей, которые очень много ездили за короткий промежуток времени. Из-за этого зависимость имеет сильно смещенное вправо нормальное распределение.***
***

In [ ]:
#Нужно ли преобразовывать?
Is_need_transform(df.query('Kaggle==0'),'mileage_per_age',lambda x: np.log(1+x))
#Преобразование
df['log_mileage_per_age'] = df['mileage_per_age'].apply(lambda x: np.log(1+x))

In [ ]:
#Информация по признаку
info_num(df,'log_mileage_per_age')

***
***С ростом пробега в месяц падает цена. Опять же, это скорее всего связано напрямую с увеличением пробега.***
***

In [ ]:
col_drp += ['Age','Age_per_owners','Владение','mileage_per_age']
col_num += ['sqr_age','sqr_age_per_owners','log_mileage_per_age']
col_bin += ['Is_NaN_Age']
col_cat += ['new_n_owners']

# 4.Analysis

***
***В данной главе содержится статистический анализ номинативных признаков, корреляционный анализ числовых и генерация новых на основе методик NLP.***
***

## 4.1) Post-analysis

In [ ]:
#Осмотр собранных признаков
col_num, col_cat, col_bin, col_drp

***
***Для начала убедимся, что мы правильно обработали числовые признаки.***
***

***
<span style='color:Red'> ***Далее приведены только те признаки, которые можно дообработать. Признаки обработанные верно - не приведены.*** </span>
***

In [ ]:
#Список столбцов для преобразования
col_num_trs = ['description_n_excla','description_n_colon','description_n_minus','volume_engine','power_engine']
#Осмотр признаков
for col in col_num_trs:
    #Нужно ли преобразовывать?
    Is_need_transform(df.query('Kaggle==0'),col,lambda x: np.log(1+x))
    #Количество уникальных признаков
    print('Количество уникальных значений: {}.'.format(df[col].nunique()))
    #Преобразование
    df['log_'+col] = df[col].apply(lambda x: np.log(1+x))
    #Правка
    col_num.remove(col)
    col_drp += [col]
    col_num += ['log_'+col]

***
***Приведенные выше преобразования позволили сократить число выбросов. Приведенные ниже преобразования позволили перевести часть числовых признаков, содержащих малое число уникальных значений и имеющих экспоненциальное распределение, в бинарные признаки.***
***

In [ ]:
#Список столбцов для преобразования
col_num_cat = ['description_n_plus','description_n_perce','description_n_quest',
               'description_n_semic','description_n_multi']
for col in col_num_cat:
    #Количество уникальных признаков
    print('Количество уникальных значений: {}.'.format(df[col].nunique()))
    #Преобразование
    df['cat_'+col] = df[col].apply(lambda x: int(2 if x > 2 else x))
    #Информация по признаку
    info_nom(df,'cat_'+col)
    #Правка
    col_num.remove(col)
    col_drp += [col]
    col_cat += ['cat_'+col]

## 4.2) NLP

***
***Составим функцию для обработки текста. Она должна включать в сети удаление мусора (знаков пунктуации, черточек, точек), токенизацию, удаление английских букв, удаление цифр и знаков пунктуации, лемматизацию и стемминг. В конце полученная форма слово проверяется на СТОП-слова. Полученный признак записывается отдельно в clean_description.***
***

In [ ]:
def clean_text(text):
    '''
    Функция для обработки текста: удаление мусора, стемминг, лемматизация.
    Вход:
    * text - исходная строка.
    Выход:
    * Конечная строка.
    '''
    new_text = []
    #Приведение текста к нормальному виду
    text = re.sub('\n', ' ', text)
    text = re.sub('\t', ' ', text)
    text = ''.join(re.findall('[a-яА-Я\s]', text))
    #Токенизация
    for token in word_tokenize(text.lower(), language="russian"):
        #Удаление английских слов
        token = ''.join([w for w in filter(delete_english.match, token)])
        #Удаление лишних букв
        new_token = ''
        for w in token:
            #Если не цифра и знак пунктуации
            if (w not in punctuation) and (w.isdigit()==False):
                new_token += w
        token = new_token
        #Лемматизация
        token = mystem.lemmatize(token)
        if token == []:
            token = ' '
        else:
            token = token[0]
        #Стемминг
        token = snowball.stem(token)
        #Проверка стоп слово и пустое слово
        if (token not in stopwords.words("russian")) and token not in [' ','\n','\t']:
            new_text += [token]
    return ' '.join(new_text)
#Очистка
df['clean_description'] = df['description'].apply(clean_text)

***
***Далее приводим все те же операции, что и в пункте 3.5. Пояснения будут привены только к тем, что отличаются.***
***

In [ ]:
#Извлекаем те же признаки, но из очищенного описания
df['len_clean_description'] = df['clean_description'].apply(len)
df['n_clean_word'] = df['clean_description'].apply(lambda x: len(x.split(' ')))
df['len_clean_word'] = df['clean_description'].apply(lambda x: np.mean([len(i) for i in x.split(' ')]))

In [ ]:
#Разница между тем, что было, и тем, что стало
df['difference_len_desc'] = df['len_description'] - df['len_clean_description']
df['difference_len_word'] = df['len_word'] - df['len_clean_word']
df['difference_n_word'] = df['n_word'] - df['n_clean_word']

In [ ]:
#Нужно ли преобразовывать?
Is_need_transform(df.query('Kaggle==0'),'difference_len_desc',lambda x: np.log(1+abs(x)))
#Преобразование
df['log_difference_len_desc'] = df['difference_len_desc'].apply(lambda x: np.log(1+abs(x)))

In [ ]:
#Нужно ли преобразовывать?
Is_need_transform(df.query('Kaggle==0'),'difference_n_word',lambda x: np.log(1+abs(x)))
#Преобразование
df['log_difference_n_word'] = df['difference_n_word'].apply(lambda x: np.log(1+abs(x)))

In [ ]:
#Нужно ли преобразовывать?
Is_need_transform(df.query('Kaggle==0'),'len_clean_description',lambda x: np.log(1+x))
#Преобразование
df['log_len_clean_description'] = df['len_clean_description'].apply(lambda x: np.log(1+x))

In [ ]:
#Нужно ли преобразовывать?
Is_need_transform(df.query('Kaggle==0'),'n_clean_word',lambda x: np.log(1+x))
#Преобразование
df['log_n_clean_word'] = df['n_clean_word'].apply(lambda x: np.log(1+x))

In [ ]:
#Информация по признаку
for name in ['len_clean_word','difference_len_word',
             'log_difference_len_desc','log_difference_n_word',
             'log_len_clean_description','log_n_clean_word']:
    info_num(df,name,show_plot=False)

***
***Видно, что часть полученных признаков имеет небольшую корреляцию с целевым признаком.***
***

In [ ]:
col_num += ['len_clean_word','difference_len_word',
             'log_difference_len_desc','log_difference_n_word',
             'log_len_clean_description','log_n_clean_word']
col_drp += ['clean_description','difference_len_desc','difference_n_word',
            'len_clean_description','n_clean_word']

***Попробуем извлечь из текста важную информацию. Наличие отдельных слов может свидетельствовать о росте цены или ее падении. Для извлечения таких слов составим коллекцию всех слов с их частотами встречи. Далее возьмем только те слова, что встречаются более 50 раз. На основе этого составим отдельные признаки, что данный прецендент содержит данное слово/слова. Определим корреляцию данного признака с целевым и отберем только те, чья корреляция по модулю будет выше 0.2.***

In [ ]:
#Список всех слов в clean_description
from collections import Counter
C = Counter()
for desc in df['clean_description']:
    desc = desc.split(' ')
    for d in desc:
        C[d] += 1

In [ ]:
#Количество упоминаний выше
number_word = 50
impor_word = []
#Отбор слов
for name,num in C.most_common():
    if num > number_word:
        impor_word += [name]
print('Количество слов: {}.'.format(len(impor_word)))

***
<span style='color:Red'> ***Количество будущих возможных признаков очень высокое.***</span>
***

In [ ]:
dfc = df.copy()
#Создание признаков
for name in impor_word:
    dfc[name] = dfc['clean_description'].apply(lambda x: sum([True for i in x.split(' ') if i in name]) > 0).astype(int)

In [ ]:
#Список столбцов для матрицы корреляции
cols = impor_word
cols += ['log_price']
#Пороговое значение
corr_min = 0.20
#Матрица корреляций
table_corr = abs(dfc[cols].dropna().corr().iloc[-1]).sort_values(ascending=False)
#Список
most_impor_word = list(table_corr[table_corr > corr_min].index)

In [ ]:
#Список слов
print(most_impor_word[1:])

***
<span style='color:Red'> ***Но это количество было успешно сокращено до 21.***</span>
***

In [ ]:
#Отсекаем целевой признак
most_impor = most_impor_word[1:]
#Создание признаков
for name in most_impor:
    df[name] = df['clean_description'].apply(lambda x: sum([True for i in x.split(' ') if i in name]) > 0).astype(int)

In [ ]:
col_bin += most_impor

## 4.3) HeatMap

***
***Для корреляционного признака составим тепловую карту и отберем те признаки, чья корреляция выше 0.95 с нецелевым, для удаления.***
***

***
<span style='color:Red'> ***В особых случаях признак оставляется, даже если его корреляция высокая. Для этого проводится тестирование с этим признаком и без него на валидационной и тестовой (Kaggle) выборках. Если с ним лучше, то он остается.*** </span>
***

In [ ]:
#Тепловая карта корреляций
heatmap_corr(df.query('Kaggle==0'),col_num+['log_price'],size=(40,40))

In [ ]:
#Правка
col_drp += ['description_n_all','description_n_space','len_description','n_word',
            'log_difference_len_desc','log_difference_n_word','log_n_clean_word']
col_num.remove('description_n_all')
col_num.remove('description_n_space')
col_num.remove('len_description')
col_num.remove('n_word')
col_num.remove('log_difference_len_desc')
col_num.remove('log_difference_n_word')
col_num.remove('log_n_clean_word')

***
***Признаки, относящиеся к clean_description полностью скоррелированы с признаками, относящемся к description, что логично. Ведь преобразования в пункте 4.2 практически не сокращают число слов (за исключением удаления стоп-слов).***
***

***
***Итоговая карта.***
***

In [ ]:
#Тепловая карта корреляций
heatmap_corr(df.query('Kaggle==0'),col_num+['log_price'],size=(30,30))

***
***Самым спорным тут является то, что признак log_price_naive сильно скоррелирован с целевым признаком. Как и говорилось ранее - это может привести к переобучению. Но выбор данного метода работы с признаком model был обоснован ранее. Тесты также показали, что при работе с признаком log_price_naive точность предсказаний на Kaggle выше.***
***

## 4.4) T-test

***
***Для теста статистической значимости признаков возьмем к-т доверия 0.95. Тест проводится с помощью метода T-Стьюдента.***
***

In [ ]:
#Статистически ли значимы ли параметры?
for col in col_bin+col_cat:
    Is_stat_dif(col,'log_price',df.query('Kaggle==0'),0.05)

***
***Как и говорилось ранее, часть признаков не значима. Удалим их.***
***

In [ ]:
#Удаление
col_drp += ['Is_NaN_Age','name_S-tronic','name_AMT',
            'cat_description_n_quest','cat_description_n_semic']
col_bin.remove('Is_NaN_Age')
col_bin.remove('name_S-tronic')
col_bin.remove('name_AMT')
col_cat.remove('cat_description_n_quest')
col_cat.remove('cat_description_n_semic')

## 4.5) F-Value

***
***С помощью ANOVA определим значимость числовых признаков.***
***

In [ ]:
plt.rcParams['figure.figsize'] = (20,10)
#Числовые признаки
imp_num = pd.Series(f_classif(df.query('Kaggle==0')[col_num], df.query('Kaggle==0')['log_price'])[0], index=col_num)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

***
***Один основной признак - наивная цена на основе комбинаций категорий модели, бренда и даты модели. Три менее значимых: год производства модели, год выпуска модели и пробег автомобиля. Далее следуют экномичность, возраст автомобиля, параметры его двигателя и описания. И так далее.***
***

***
<span style='color:Red'> ***К сожалению не хватило времени, чтобы поработать над удалением не значимых числовых признаков и обучении модели без них. Лучше или хуже будет результат?*** </span>
***

# 5.ML

## 5.1) Prepare

***
***Перед машинным обучением разделим на обучающую, валидационную и тестовую выборки. Применим стандартную нормализацию и dummy кодирование к номинативным признакам.***
***

In [ ]:
#Разделение выборок
train = df.query('Kaggle==0').drop(columns=col_drp+['Kaggle']).copy()
test  = df.query('Kaggle==1').drop(columns=col_drp+['Kaggle','log_price']).copy()

In [ ]:
#Стандартизация
scaler = StandardScaler()
scaler.fit(train[col_num])
train.loc[:,col_num] = scaler.transform(train[col_num])
test.loc[:,col_num]  = scaler.transform(test[col_num])

In [ ]:
#Кодирование
train = pd.concat([train[col_num+['log_price']],pd.get_dummies(train[col_bin+col_cat].astype(str))],axis=1)
test = pd.concat([test[col_num], pd.get_dummies(test[col_bin+col_cat].astype(str))],axis=1)

In [ ]:
#Проверка на наличие признаков
for col in train:
    if col not in test and col != 'log_price':
        print('В тестовой выборке отсутствует признак: {}'.format(col))
        test[col] = 0

In [ ]:
#Выделение целевого признака
X = train.drop(columns=['log_price'])
Y = train['log_price']

In [ ]:
#Правка массива
test = test[X.columns]

In [ ]:
#Разделение выборок
X_trn,X_val,Y_trn,Y_val = train_test_split(X,Y,test_size=0.15,shuffle=True,random_state=random_state)

In [ ]:
#Проверка
sum_col = len(col_num)
for col in col_cat+col_bin:
    sum_col += df[col].nunique()
print('Число столбцов, которое должно быть: {}. Число столбцов в train: {}, test: {}.'.\
      format(sum_col,len(train.columns)-1,len(test.columns)))

## 5.2) CatBoost

***
***Как следует из описания к проекту на SkillFactory, стоит попробовать сначала алгоритм CatBoost. Для определения его гиперпараметров воспользуемся двумя методами: встроенной функцией в библиотеке catboost - gridsearch с кросс-валидацией cv=5, и собственным gridsearch. Почему-то последний показал лучшее качество работы. Оптимальной считается та модель, чья метрика признака price (не log_price) лучше на валидационной выборке.*** 
***

***
***Для сокращении времени расчета на Kaggle, эта часть кода закомментирована и обучается оптимальная модель.***
***

In [ ]:
'''
mas = []
i = 0
#Оптимизация через свой GridSearch
for lr in [0.03,0.05,0.10]:
    for d in [5,6,7,8]:
        for rs in [0.5,0.7,0.9]:
            for l2 in [1,6,11]:
                #Модель
                model = CatBoostRegressor(iterations = 7500,
                                          learning_rate = lr,
                                          l2_leaf_reg = l2,
                                          depth = d,
                                          min_data_in_leaf = 1,
                                          random_strength = rs,
                                          random_seed = random_state,
                                          eval_metric = 'MAPE',
                                          od_wait = 300
                                         )
                #Обучение
                model.fit(X_trn, Y_trn,
                          eval_set=(X_val, Y_val),
                          verbose_eval=1000,
                          use_best_model=True
                         )
                #Предсказание
                Y_pred_trn = model.predict(X_trn)
                Y_pred_val = model.predict(X_val)
                #Метрика
                mape_trn = mape(Y_trn,Y_pred_trn)
                mape_val = mape(Y_val,Y_pred_val)
                mape_trn_exp = mape(np.exp(Y_trn),np.exp(Y_pred_trn))
                mape_val_exp = mape(np.exp(Y_val),np.exp(Y_pred_val))
                #Вывод информации
                mas += [[i,lr,d,rs,l2,mape_trn,mape_val,mape_trn_exp,mape_val_exp]]
                i += 1
                print(i,lr,d,rs,l2,mape_trn,mape_val,mape_trn_exp,mape_val_exp)
result = pd.DataFrame(mas,columns=['i','lr','d','rs','l2','mape_trn','mape_val','mape_trn_exp','mape_val_exp'])
result.iloc[[result['mape_trn_exp'].argmin(),result['mape_val_exp'].argmin()],:]
'''
print('Удалено.')

***
***Можно заметить, что модель обучается не на обучающей выборке, а на всей. Были проведены тесты сабмитов на Kaggle для данной модели, обученных на той или иной выборке и лучшие результаты (на 0.5-1.0%) показали те сабмиты, что были обучены на всей выборке, а не только на обучающей.***
***

In [ ]:
#Модель - оптимальная по валидационной выборке
model_cat = CatBoostRegressor(
    iterations = 5000,              #Максимальное число итераций
    learning_rate = 0.03,           #Темп обучения
    l2_leaf_reg = 6,                #К-т при регуляризации
    depth = 7,                      #Максимальная глубина деревьев
    min_data_in_leaf = 1,           #Минимальное число прецендентов в лепестке
    random_strength = 0.7,          #Процент от выборки для валидации
    random_seed = random_state,     #Воспроизведение результатов
    eval_metric = 'MAPE',           #Метрика
    od_wait = 300                   #Количество итераций после достижения оптимума
)
#Обучение
model_cat.fit(
    X, Y,
    eval_set=(X_val, Y_val),
    verbose_eval=100,
    use_best_model=True
)

In [ ]:
#Словарь с результатами
dict_res = model_cat.evals_result_
df_res = pd.DataFrame({'trn':dict_res['learn']['MAPE'],'val':dict_res['validation']['MAPE']}) * 100
#График
df_res.plot(figsize=(20,8))

In [ ]:
#Предсказание
Y_pred_trn = model_cat.predict(X_trn).flatten()
Y_pred_val = model_cat.predict(X_val).flatten()
#Метрика
mape_trn = mape(np.exp(Y_trn),np.exp(Y_pred_trn))
mape_val = mape(np.exp(Y_val),np.exp(Y_pred_val))
print('Метрика MAPE на обучающей выборке: {:.3f}%, валидационной выборке: {:.3f}%.'.format(mape_trn,mape_val))

***
***В сравнении с baseline решением точность возросла с 19.88% до ~3.2%.***
***

In [ ]:
#Сохранение на Kaggle
predict_to_Kaggle(model_cat,test,name='submission_cat')

***
***Точность на Kaggle данного сабмита - 11.402%. 15/55 место. Уже неплохо.***
***

## 5.3) XGBoost

***
***Попробуем применить знания, полученные ранее. Обучим несколько моделей и создадим их ансамбль. В качестве второй модели возьмем небезизвестную XGBoost.***
***

***
***Аналогично снизу кусок кода по поиску гиперпараметров модели.***
***

In [ ]:
'''
mas = []
i = 0
#Оптимизация через свой GridSearch
for lr in [0.03,0.05,0.10]:
    for d in [6,7,8]:
        for rs in [0.5,0.65,0.8]:
            for rc in [0.5,0.65,0.8]:
                for l2 in [0.8,1.0,1.2]:
                    for ne in [400,600,800,1000]:
                        #Модель
                        model = XGBRegressor(
                            random_state=random_state,n_jobs=-1,
                            learning_rate=lr,
                            n_estimators=ne,max_depth=d,
                            subsample=rs,colsample_bytree=rc,alpha=l2
                        )
                        #Обучение
                        model.fit(X_trn,Y_trn,
                                 eval_set=[(X_trn, Y_trn),(X_val, Y_val)],
                                 eval_metric='mape',
                                 verbose=False)
                        #Предсказание
                        Y_pred_trn = model.predict(X_trn)
                        Y_pred_val = model.predict(X_val)
                        #Метрика
                        mape_trn = mape(Y_trn,Y_pred_trn)
                        mape_val = mape(Y_val,Y_pred_val)
                        mape_trn_exp = mape(np.exp(Y_trn),np.exp(Y_pred_trn))
                        mape_val_exp = mape(np.exp(Y_val),np.exp(Y_pred_val))
                        #Вывод информации
                        mas += [[i,lr,d,rs,rc,l2,ne,mape_trn,mape_val,mape_trn_exp,mape_val_exp]]
                        i += 1
                        print(i,lr,d,rs,rc,l2,ne,mape_trn,mape_val,mape_trn_exp,mape_val_exp)
result = pd.DataFrame(mas,columns=['i','lr','d','rs','rc','l2','ne','mape_trn','mape_val','mape_trn_exp','mape_val_exp'])
'''
print('Удалено.')

***
***Аналогично обучаем на всей выборке. Только тут качество выше не на 0.5-1.0%, а всего на 0.2%.***
***

In [ ]:
#Модель - оптимальная по валидационной выборке
model_xgb = XGBRegressor(
    random_state=random_state,n_jobs=-1,
    learning_rate=0.03,
    n_estimators=800,
    max_depth=6,
    subsample=0.80,
    colsample_bytree=0.65,
    alpha=1.0
)
#Обучение
model_xgb.fit(
    X,Y,
    eval_set=[(X_trn, Y_trn),(X_val, Y_val)],
    eval_metric='mape',
    verbose=True
)

In [ ]:
#Предсказание
Y_pred_trn = model_xgb.predict(X_trn).flatten()
Y_pred_val = model_xgb.predict(X_val).flatten()
#Метрика
mape_trn = mape(np.exp(Y_trn),np.exp(Y_pred_trn))
mape_val = mape(np.exp(Y_val),np.exp(Y_pred_val))
print('Метрика MAPE на обучающей выборке: {:.3f}%, валидационной выборке: {:.3f}%.'.format(mape_trn,mape_val))

***
***Качество в 2 раза хуже, чем для CatBoost. Дальнейшее увеличение числа деревьев не привело к сильному повышению качества модели.***
***

In [ ]:
#Сохранение на Kaggle
predict_to_Kaggle(model_xgb,test,name='submission_xgb')

***
***Точность на Kaggle данного сабмита - 11.564%. 18/55 место. Стало чуть хуже.***
***

## 5.4) ensemble

***
***Рассматривались также модели RandomForest и SVM, но они дали результат на Kaggle ~12-13%. Поэтому для ансамблирования принято решения взять две лучшие модели из тестируемых ранее.***
***

In [ ]:
#Модель
estimators = [
    ('CatBoost', CatBoostRegressor(
        iterations = 5000,              #Максимальное число итераций
        learning_rate = 0.03,           #Темп обучения
        l2_leaf_reg = 6,                #К-т при регуляризации
        depth = 7,                      #Максимальная глубина деревьев
        min_data_in_leaf = 1,           #Минимальное число прецендентов в лепестке
        random_strength = 0.7,          #Процент от выборки для валидации
        random_seed = random_state,     #Воспроизведение результатов
        eval_metric = 'MAPE',           #Метрика
        od_wait = 300                   #Количество итераций после достижения оптимума
)),
    ('BaggingBoosting', XGBRegressor(
        random_state=random_state,n_jobs=-1,
        learning_rate=0.03,                           #Темп обучения
        n_estimators=800,                             #Количество деревьев
        max_depth=6,                                  #Максимальная глубина
        subsample=0.80,                               #Бутстрэп
        colsample_bytree=0.65,                        #Количество признаков для каждого дерева
        alpha=1.0                                     #Коэффициент при регуляризации
))]
#Построение моделей
model_ens = StackingRegressor(
    estimators=estimators,
    final_estimator=LinearRegression(n_jobs=-1),
    n_jobs=-1,cv=5
)

***
***Обучение происходит с K-Fold (5) кросс-валидацией, поэтому смело обучаем на всей выборке.***
***

In [ ]:
#Обучение модели
model_ens.fit(X,Y)

In [ ]:
#Предсказание
Y_pred_trn = model_ens.predict(X_trn).flatten()
Y_pred_val = model_ens.predict(X_val).flatten()
#Метрика
mape_trn = mape(np.exp(Y_trn),np.exp(Y_pred_trn))
mape_val = mape(np.exp(Y_val),np.exp(Y_pred_val))
print('Метрика MAPE на обучающей выборке: {:.3f}%, валидационной выборке: {:.3f}%.'.format(mape_trn,mape_val))

***
***Метрика хуже, чем на CatBoost. Скорее всего все портит модель XGBoost. Возможно стоит взять другую мета-модель, а возможно добавить пару других моделей для увеличения числа размерностей.***
***

In [ ]:
#Сохранение на Kaggle
predict_to_Kaggle(model_ens,test,name='submission_ens')

***
***Точность на Kaggle данного сабмита - 11.738%. 22/55 место. Стало еще хуже. Не в том направлении двигаемся.***
***

## 5.5) Analysis

***
***На данный момент лучший результат дала модели CatBoost. Возьмем ее за новое baseline решение. Известно, что модели основанные на деревьях решений не могут дать выбросы целевого признака (аномально большие или аномально малые), так как они ограничены числовыми значениями прецендентов. Но нейронные сети от этого не застрахованы и полученные предсказания необходимо пост-обрабатывать, чтобы удалять выбросы, обрубая их под некоторые максимально и минимально допустимые. Такими границами будут являться предикты модели CatBoost.***
***

In [ ]:
#Предсказание
Y_pred_trn = model_cat.predict(X_trn).flatten()
Y_pred_val = model_cat.predict(X_val).flatten()
#Метрика
mape_trn = mape(np.exp(Y_trn),np.exp(Y_pred_trn))
mape_val = mape(np.exp(Y_val),np.exp(Y_pred_val))
print('Метрика MAPE на обучающей выборке: {:.3f}%, валидационной выборке: {:.3f}%.'.format(mape_trn,mape_val))

In [ ]:
print('Максимальное значение предсказанное:  {:.3f}.'.format(np.hstack([Y_pred_trn,Y_pred_val]).max()))
print('Максимальное значение действительное: {:.3f}.'.format(np.hstack([Y_trn,Y_val]).max()))
print('Минимальное  значение предсказанное:  {:.3f}.'.format(np.hstack([Y_pred_trn,Y_pred_val]).min()))
print('Минимальное  значение действительное: {:.3f}.'.format(np.hstack([Y_trn,Y_val]).min()))

In [ ]:
#Ограничиваем предсказания для NN
max_pred = np.hstack([Y_trn,Y_val]).max()
min_pred = np.hstack([Y_trn,Y_val]).min()

***
***Создадим новые функции для обучения NN и обновим старые в соответствии с описанным выше условием.***
***

In [ ]:
def current_pred(mas):
    #Функция для отсечения выбросов
    for i,m in enumerate(mas):
        if max_pred < m:
            mas[i] = max_pred
        elif min_pred > m:
            mas[i] = min_pred
    return mas

def predict_to_Kaggle(model,X_test,name='submission'):
    '''
    Сохранение результатов на Kaggle.
    Вход:
    * model - обученная модель;
    * X_test - матрица прецендентов для предсказания;
    * name - название файла.
    Выход:
    * None.
    '''
    #Предсказание на test
    Y_pred_test = model.predict(X_test)
    #Правка
    Y_pred_test = current_pred(Y_pred_test)
    #Пост-обработка - удаление логорифмирования целевого признака
    Y_pred_test = np.exp(Y_pred_test)
    #Создание DF для вывода
    sample_submission = pd.read_csv(PATH + 'sample_submission.csv')
    sample_submission['price'] = Y_pred_test
    #Вывод в файл
    sample_submission.to_csv(name+'.csv', index=False)
    pass

def predict_trn_val(model,X_trn,X_val,Y_trn,Y_val):
    '''
    Предсказания модели на обучающей и валидационной выборках.
    Вход:
    * model - обученная модель;
    * X_trn,X_val,Y_trn,Y_val - выборки.
    Выход:
    * None.
    '''
    #Предсказание
    Y_pred_trn = model.predict(X_trn).flatten()
    Y_pred_val = model.predict(X_val).flatten()
    #Правка
    Y_pred_trn = current_pred(Y_pred_trn)
    Y_pred_val = current_pred(Y_pred_val)
    #Метрика
    mape_trn = mape(np.exp(Y_trn),np.exp(Y_pred_trn))
    mape_val = mape(np.exp(Y_val),np.exp(Y_pred_val))
    print('Метрика MAPE на обучающей выборке: {:.3f}%, валидационной выборке: {:.3f}%.'.format(mape_trn,mape_val))
    pass

def get_plot_NN(history):
    #Построение графика функции потерь (mape) по итерациям
    fig, ax = plt.subplots(figsize=(20,10))
    plt.title('Loss for NN - log10(MAPE)')
    plt.plot(np.log10(history.history['MAPE']), label='train')
    plt.plot(np.log10(history.history['val_MAPE']), label='val')
    ax.legend()
    plt.show()
    pass

def create_callbacks_list(patience_ES,patience_RLROP,verbose=0):
    '''
    Сохранение callbacks для модели NN.
    Вход:
    * patience_ES - число ожиданий эпох падения метрики на валидационной выборке до отключения обучения;
    * patience_RLROP - число ожиданий эпох падения метрики на валидационной выборке до понижения темпа обучения;
    * verbose - выводить ли инфрмацию?
    Выход:
    * Callbacks.
    '''
    ### Сохранение прогресса обучения
    #Сохранение модели
    checkpoint = ModelCheckpoint('best_model.hdf5',
                                 monitor='val_MAPE',
                                 verbose=verbose,
                                 mode='auto',
                                 save_best_only=True)
    #Ранняя оставка, когда метрика не растет
    earlystop = EarlyStopping(monitor='val_MAPE',
                              patience=patience_ES,
                              restore_best_weights=True)
    #Уменьшение темпа обучения, когда метрика падает
    reduce_lr = ReduceLROnPlateau(monitor='val_MAPE',           #Метрика контроля
                                  factor=0.1,                   #Во сколько раз снижается
                                  patience=patience_RLROP,      #Кол-во эпох без улучшения
                                  min_lr=0.0000001,             #Минимальное значение
                                  verbose=verbose,              #Вывод информации
                                  mode='auto')                  #Какая величина контролируется
    #Полный список callbacks
    callbacks_list = [checkpoint, earlystop, reduce_lr]
    return callbacks_list

# 6.NN

## 6.1) MLP

***
***Сначала попробуем обучить обычный многослойный перцептрон (в дальнейшем MLP). Для этого необходимо выбрать правильную архитектуру и настройки сети (своего рода гиперпараметры). Напишем для этого отдельные функции поиска этих оптимальных параметров.***
***

In [ ]:
'''
def create_NN(lr,bs,ep,number,act,do,bn,verbose=1):
    #Фунция для создания и обучения сети.
    #Вход:
    #* lr - темп обучения;
    #* bs - размер батча;
    #* ep - количество эпох;
    #* number - число нейронов в каждом из слоев;
    #* act - виды функций активаций в каждом из слоев;
    #* do - число отключаемых нейронов в каждом из слоев;
    #* bn - батч-нормализация;
    #* verbose - вывод информации.
    #Выход:
    #* Метрика MAPE на валидационной и обучающей выборках.
    #Формирование модели
    model = Sequential(name='MLP')
    for i,n,a,d,b in zip(range(len(number)),number,act,do,bn):
        if i == 0:
            model.add(L.Dense(n, input_dim=X_trn.shape[1], activation=a))
            if b==1:
                model.add(L.BatchNormalization())
            model.add(L.Dropout(d, seed=random_state))
        elif n==1:
            model.add(L.Dense(n, activation=a))
        else:
            model.add(L.Dense(n, activation=a))
            if b==1:
                model.add(L.BatchNormalization())
            model.add(L.Dropout(d, seed=random_state))
    #Информация о собранной модели
    model.summary()
    #Компиляция задачи: модель, метрика и функция потерь
    model.compile(loss='MAPE', 
                  optimizer=O.Adam(lr=lr), 
                  metrics=['MAPE'])
    #Обучение
    history = model.fit(
        X_trn, Y_trn,
        batch_size=bs,
        epochs=ep,
        validation_data=(X_val, Y_val),
        callbacks=create_callbacks_list(50,30,verbose=0),
        verbose=verbose
    )
    #Загрузка лучшей модели
    model.load_weights('../working/best_model.hdf5')
    #Предсказание
    Y_pred_trn = model.predict(X_trn).flatten()
    Y_pred_val = model.predict(X_val).flatten()
    #Метрика
    mape_trn = mape(Y_trn,Y_pred_trn)
    mape_val = mape(Y_val,Y_pred_val)
    mape_trn_exp = mape(np.exp(Y_trn),np.exp(Y_pred_trn))
    mape_val_exp = mape(np.exp(Y_val),np.exp(Y_pred_val))
    #Вывод
    return model,mape_trn,mape_val,mape_trn_exp,mape_val_exp

mas = []
#Оптимизация гиперпараметров сети
for N in [3,4,5,6]:
    for st in [512,1024]:
        for ACT in ['elu']:
            for DO in [0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7]:
                for BN in [1]:
                    #Составление параметров для передачи в функцию
                    number = []
                    for i in range(N-1):
                        number += [st/(2**i)]
                    number += [1]
                    act = [ACT] * (N-1) + ['linear']
                    do =  [DO] * N
                    bn = [BN] * N
                    #Вывод информации
                    print(number, act, do, bn)
                    #Обучение
                    model,trn,val,trn_exp,val_exp = create_NN(
                        lr=0.1,bs=512,ep=1000,
                        number=number,
                        act=act,
                        do=do,
                        bn=bn,
                        verbose=0
                    )
                    #Запись
                    mas += [[number,act,do,bn,trn,val,trn_exp,val_exp]]  
'''
print('Удалено.')

***
***Эксперименты показали, что сеть с батч-нормализацией работает на порядок лучше и сходиться быстрее. Также в качестве функции активации стоит брать функцию elu для НЕвыходного слоя (тестировались relu,,elu,selu). Количество слоев и число нейронов было подобрано из алгоритма выше. Также из этого алгоритма подобрано число связей нейронов, отключаемых слоями DropOut.***
***

In [ ]:
#Формирование модели - оптимальной
model_mlp_alone = Sequential(name='MLP')
#Первый слой
model_mlp_alone.add(L.Dense(512, input_dim=X_trn.shape[1], activation='elu'))
model_mlp_alone.add(L.BatchNormalization())
model_mlp_alone.add(L.Dropout(0.35, seed=random_state))
#Второй слой
model_mlp_alone.add(L.Dense(256, activation='elu'))
model_mlp_alone.add(L.BatchNormalization())
model_mlp_alone.add(L.Dropout(0.35, seed=random_state))
#Третий слой
model_mlp_alone.add(L.Dense(128, activation='elu'))
model_mlp_alone.add(L.BatchNormalization())
model_mlp_alone.add(L.Dropout(0.35, seed=random_state))
#Четвертый слой
model_mlp_alone.add(L.Dense(1, activation='linear'))
#Информация о собранной модели
model_mlp_alone.summary()

***
***Получилась маленькая сеть на 243 000 обучаемых параметров. Дальнейшее увеличение числа слоев или нейронов приводило к постепенному уменьшению метрики на валидации. А уменьшение слоев к резкому падению метрики.***
***

***
<span style='color:Red'> ***Здесь и далее, как и говорилось в самом начале, я не знаю почему модель не является воспроизводимой, я вроде зафиксировал все random_seed. Поэтому лучший сабмит будет загружен на Git-Hub (ссылка в конце работы). Также, так как результаты не являются воспроизводимыми, я буду описывать не то, что Вы увидите, а то, что видел я - сами сабмиты и выводы по обучению и его ходу.*** </span>
***

In [ ]:
#Компиляция задачи: модель, метрика и функция потерь
model_mlp_alone.compile(
    loss='MAPE', 
    optimizer=O.Adam(lr=0.1), 
    metrics='MAPE'
)
#Обучение
history = model_mlp_alone.fit(
    X_trn,Y_trn,
    batch_size=512,
    epochs=5000,
    validation_data=(X_val,Y_val),
    callbacks=create_callbacks_list(150,120,verbose=1),
    verbose=1
)

***
***Числа 150 и 120 в callbacks были также подобраны опытным путем. Изначально стояли 50 и 30. Но эксперименты показали, что сети "надо дать шанс" и увеличить количество эпох без повышения метрики на валидации. Это позволило повысить точность результатов на Kaggle с ~12.5% до ~11.8%. Темп обучения также выбирался между 0.1, 0.05 и 0.01. Оптимизер между Adam и Nadam. Лучший вариант приведен выше.***
***

In [ ]:
#График метрик
get_plot_NN(history)

***
***Все результаты сохраняем.***
***

In [ ]:
#Загрузка лучшей модели
model_mlp_alone.load_weights('best_model.hdf5')
#Сохранение модели
model_mlp_alone.save('MLP.hdf5')

In [ ]:
#Предсказание на train, valid
predict_trn_val(model_mlp_alone,X_trn,X_val,Y_trn,Y_val)

In [ ]:
#Сохранение на Kaggle
predict_to_Kaggle(model_mlp_alone,test,name='submission_mlp')

***
***Точность на Kaggle данного сабмита - 11.888%. 28/55 место. Хуже, чем у классического ML, но есть над чем работать.***
***

## 6.2) MLP + NLP

***
***Попробуем объединить полученную ранее архитектуру с RNN сетью (задача NLP).***
***

In [ ]:
#Распределение количества слов
df['clean_description'].apply(lambda x: len(x.split())).hist(bins=50,figsize=(20,8))
#Кввантиль 0.95
print('Квантиль 0.95: {:.3f}'.format(df['clean_description'].apply(lambda x: len(x.split())).quantile(0.95)))

***
***Необходимо определиться сколько каждый прецендент может хранить максимальное число токенов. Возьмем 0.95 квантиль от распределений числа токенов по прецендентам и округлим до n-ой степени двойки. Число 246->256.*** 
***

In [ ]:
#Максимальное количество слов со всех прецендентов
MAX_WORDS = 100000
#Максимальное количество слов в последовательности
MAX_SEQUENCE_LENGTH = 256

In [ ]:
#Разделение выборок и выделение признака description, содержащего текст 
train_text = df.query('Kaggle==0')['clean_description']
test_text  = df.query('Kaggle==1')['clean_description']
#Разделение обучающей на обучающую и валидационную
trn_text = train_text[X_trn.index]
val_text = train_text[X_val.index]
#Токенизация
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(df['clean_description'])
#Перевод текста с помощью токенов в последовательность
trn_text_seq   = sequence.pad_sequences(tokenize.texts_to_sequences(trn_text),  maxlen=MAX_SEQUENCE_LENGTH)
val_text_seq   = sequence.pad_sequences(tokenize.texts_to_sequences(val_text),  maxlen=MAX_SEQUENCE_LENGTH)
test_text_seq  = sequence.pad_sequences(tokenize.texts_to_sequences(test_text), maxlen=MAX_SEQUENCE_LENGTH)

***
***Текст перевели в последовательность, каждое число, это закодированное слово (из очищенного признака clean_description).***
***

In [ ]:
#Формирование RNN сети
model_nlp = Sequential(name='NLP')
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH))
#Первый слой
model_nlp.add(L.LSTM(256, return_sequences=True))
model_nlp.add(L.Dropout(0.25, seed=random_state))
#Второй слой
model_nlp.add(L.LSTM(128))
model_nlp.add(L.Dropout(0.25, seed=random_state))
#Третий слой
model_nlp.add(L.Dense(64, activation='elu'))
model_nlp.add(L.BatchNormalization())
model_nlp.add(L.Dropout(0.25, seed=random_state))
#Четвертый слой
model_nlp.add(L.Dense(32, activation='elu'))
model_nlp.add(L.BatchNormalization())
model_nlp.add(L.Dropout(0.25, seed=random_state))

***
***Архитектура RNN сети также подбиралась опытным путем. Были попытки заменить слой LSTM на GRU - стало значительно хуже. Число нейронов и число Dense слоев подобрано опытным путем.***
***

In [ ]:
#Формирование MLP сети
model_mlp = Sequential(name='MLP')
#Первый слой
model_mlp.add(L.Dense(512, input_dim=X_trn.shape[1], activation='elu'))
model_mlp.add(L.BatchNormalization())
model_mlp.add(L.Dropout(0.35, seed=random_state))
#Второй слой
model_mlp.add(L.Dense(256, activation='elu'))
model_mlp.add(L.BatchNormalization())
model_mlp.add(L.Dropout(0.35, seed=random_state))
#Третий слой
model_mlp.add(L.Dense(128, activation='elu'))
model_mlp.add(L.BatchNormalization())
model_mlp.add(L.Dropout(0.35, seed=random_state))

***
***Архитектура MLP сети та же.***
***

In [ ]:
#Группировка сети RNN+MLP
combinedInput = L.concatenate([model_nlp.output, model_mlp.output])
#Добавление новых слоев
#Первый слой
head = L.Dense(64, activation='elu')(combinedInput)
head = L.BatchNormalization()(head)
#Выходной
head = L.Dense(1, activation='linear')(head)
model_rnn_mlp = M(inputs=[model_nlp.input,model_mlp.input], outputs=head)
#Информация о собранной модели
model_rnn_mlp.summary()

***
***Архитектура головы также подбиралась. Была попытка увеличить число слоев или число нейронов. Это не дало никакого значимого улучшения результатов.***
***

***
***Для лучшей сходимости сети понизим темп обучения в сравнении с обучением MLP в 10 раз. Также тут были попытки сначала обучить MLP сеть, заморозив веса RNN сети, а потом дообучить все сразу, но это давало такой же результат, но занимало при этом больше времени.***
***

In [ ]:
#Компиляция задачи: модель, метрика и функция потерь
model_rnn_mlp.compile(
    loss='MAPE', 
    optimizer=O.Adam(lr=0.01), 
    metrics='MAPE'
)
#Обучение
history = model_rnn_mlp.fit(
    [trn_text_seq, X_trn], Y_trn,
    batch_size=512,
    epochs=1000,
    validation_data=([val_text_seq, X_val], Y_val),
    callbacks=create_callbacks_list(150,120,verbose=1),
    verbose=1
)

In [ ]:
#График метрик
get_plot_NN(history)

In [ ]:
#Загрузка лучшей модели
model_rnn_mlp.load_weights('../working/best_model.hdf5')
#Сохранение модели
model_rnn_mlp.save('../working/MLP_NLP.hdf5')

In [ ]:
#Предсказание на train, valid
predict_trn_val(model_rnn_mlp,[trn_text_seq, X_trn],[val_text_seq, X_val],Y_trn,Y_val)

In [ ]:
#Сохранение на Kaggle
predict_to_Kaggle(model_rnn_mlp,[test_text_seq,test],name='submission_nlp')

***
***Точность на Kaggle данного сабмита - 11.800%. 25/55 место. Хуже, чем у классического ML, но лучше, чем у MLP. То есть добавление RNN сети позволило улучшить качество результата. Попробуем добавить еще CNN сеть.***
***

## 6.3) MLP + NLP + CV

***
***Добавим CNN сеть для решения задачи компьютерного зрения (в дальнейшем CV) по распознованию картинок (фото) продаваемых автомобилей.***
***

***
***Посмотрим на картинки.***
***

In [ ]:
#Размер картинки
plt.figure(figsize = (18,12))
#Случайные nxn картинок
n = 5
random_image = df.query('Kaggle==0').sample(n = n**2)
random_image_paths = random_image['sell_id'].values
random_image_cat = random_image['price'].values
#Вывод
for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(PATH+'img/img/' + str(path) + '.jpg')
    plt.subplot(n,n,index+1)
    plt.imshow(im)
    plt.title('price: '+str(int(random_image_cat[index]))+' руб.')
    plt.axis('off')
plt.show()

***
***Автомобили сфотографированы с разных ракурсов при разном освещении на разного качества фотоаппараты. Это важный момент, который необходимо учесть при аугментации.***
***

In [ ]:
#Разделение выборок и выделение признака description, содержащего текст 
train_image = df.query('Kaggle==0')['sell_id']
test_image  = df.query('Kaggle==1')['sell_id']
#Разделение обучающей на обучающую и валидационную
trn_image = train_image[X_trn.index]
val_image = train_image[X_val.index]

***
***Извлечем картинки из данных.***
***

In [ ]:
#Размер картинки
size = (320, 240)
#Функция извлечения картинок
def get_image_array(data):
    images_train = []
    for index, sell_id in enumerate(data):
        image = cv2.imread(PATH + 'img/img/' + str(sell_id) + '.jpg')
        assert(image is not None)
        image = cv2.resize(image, size)
        images_train.append(image)
    images_train = np.array(images_train)
    print('images shape', images_train.shape, 'dtype', images_train.dtype)
    return(images_train)
#Извлечение
trn_img  = get_image_array(trn_image.values)
val_img  = get_image_array(val_image.values)
test_img = get_image_array(test_image.values)

***
***Опытным путем была установлена данная комбинация аугментаций. Были удалены обработки, меняющие цвет автомобиля (ведь цвет влияет на конечную цену).***
***

In [ ]:
#Аугментация
p = 0.5 #Вероятность изменить изображение
#Создание аугментатора
augment = alb.Compose([
    #Размытие изображения
    alb.GaussianBlur(p=p),
    #Добавление шума
    alb.GaussNoise(p=p),
    #Поворот вокруг вертикальной оси
    alb.HorizontalFlip(p=p),
    #Изменение яркости
    alb.RandomBrightness(p=p,limit=(0.1,0.3)),
    #Изменение контраста
    alb.RandomContrast(p=p,limit=(0.1,0.3)),
    #Вращение изображения
    alb.ShiftScaleRotate(shift_limit=0.0625,      #Коэффициент изменения сдвига
                         scale_limit=(0.1,0.2),   #Коэффициент изменение масштаба
                         interpolation=1,         #Флаг для вида интерполяции (линейная)
                         border_mode=4,           #Флаг для экстраполяцц (отражение)
                         rotate_limit=20,         #Угол поворота
                         p=0.7),                  #Вероятность
])

***
***Посмотрим в качество примера, как работает аугментация.***
***

In [ ]:
#Размер картинки
plt.figure(figsize = (18,12))
#Вывод пример аугментации
for i in range(n**2):
    img = augment(image = trn_img[1])['image']
    plt.subplot(n,n,i+1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

***
<span style='color:Red'> ***Повороты, шум, контрасность, освещение и так далее.*** </span>
***

In [ ]:
#Функция для аугментации изображений
def process_image(image):
    return augment(image = image.numpy())['image']

#Фукнция для обучающей и валидационной выборках
def tf_process_trnval_dataset_element(image, table_data, text, price):
    im_shape = image.shape
    [image,] = tf.py_function(process_image, [image], [tf.uint8])
    image.set_shape(im_shape)
    return (image, table_data, text), price

In [ ]:
#Обучающая выборка
trn_dataset = tf.data.Dataset.from_tensor_slices((
    trn_img,
    X_trn,
    trn_text_seq,
    Y_trn
    )).map(tf_process_trnval_dataset_element)
#Валидационная выборка
val_dataset = tf.data.Dataset.from_tensor_slices((
    val_img,
    X_val,
    val_text_seq,
    Y_val
    )).map(tf_process_trnval_dataset_element)
#Тестовая выборка
test_dataset = tf.data.Dataset.from_tensor_slices((
    test_img,
    test,
    test_text_seq,
    np.zeros(len(test))
    ))

***
***Проверим, что все правильно, код запускается.***
***

In [ ]:
#Проверка
trn_dataset.__iter__().__next__();
val_dataset.__iter__().__next__();
test_dataset.__iter__().__next__();

***
***Все запускается.***
***

***
***Как было показано в предыдущем проекте, при обучении сетей CNN для решения задач CV, часто пользуются заранее предобученными моделями. Они созданы для решения других задач, но комбинации их весов позволяют извлекать из изображений уникальные полезные признаки, для которых получение тут с нуля потребовалось бы много-много дней счета, чего у студента нету. Берем предобученную модель EfficientNetB3 или ...B4. Эксперименты показали, что разницы никакой, поэтому берем ту, которая обучается быстрее - B3.***
***

In [ ]:
#Базовая модель - голова
model_img = efn.EfficientNetB3(weights='imagenet',               #Обученная на imagenet
                               include_top=False,                #Включать ли верхнюю часть сети
                               input_shape=(size[1],size[0],3))  #Размер матрицы
#Формирование CV сети
model_cv = Sequential(name='CV')
model_cv.add(model_img)
model_cv.add(L.GlobalAveragePooling2D())

***
***Далее добавляем уже известные архитектуры RNN и MLP сетей, объединяем и запускаем это.***
***

In [ ]:
#Формирование RNN сети
model_nlp = Sequential(name='NLP')
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH))
#Первый слой
model_nlp.add(L.LSTM(256, return_sequences=True))
model_nlp.add(L.Dropout(0.25, seed=random_state))
#Второй слой
model_nlp.add(L.LSTM(128))
model_nlp.add(L.Dropout(0.25, seed=random_state))
#Третий слой
model_nlp.add(L.Dense(64, activation='elu'))
model_nlp.add(L.BatchNormalization())
model_nlp.add(L.Dropout(0.25, seed=random_state))
#Четвертый слой
model_nlp.add(L.Dense(32, activation='elu'))
model_nlp.add(L.BatchNormalization())
model_nlp.add(L.Dropout(0.25, seed=random_state))

In [ ]:
#Формирование MLP сети
model_mlp = Sequential(name='MLP')
#Первый слой
model_mlp.add(L.Dense(512, input_dim=X_trn.shape[1], activation='elu'))
model_mlp.add(L.BatchNormalization())
model_mlp.add(L.Dropout(0.35, seed=random_state))
#Второй слой
model_mlp.add(L.Dense(256, activation='elu'))
model_mlp.add(L.BatchNormalization())
model_mlp.add(L.Dropout(0.35, seed=random_state))
#Третий слой
model_mlp.add(L.Dense(128, activation='elu'))
model_mlp.add(L.BatchNormalization())
model_mlp.add(L.Dropout(0.35, seed=random_state))

In [ ]:
#Группировка сети RNN+MLP+CV
combinedInput = L.concatenate([model_cv.output, model_mlp.output, model_nlp.output])
#Добавление новых слоев
#Первый слой
head = L.Dense(64, activation='elu')(combinedInput)
head = L.BatchNormalization()(head)
#Выходной
head = L.Dense(1, activation='linear')(head)
model_rnn_mlp_cv = M(inputs=[model_cv.input,model_mlp.input,model_nlp.input], outputs=head)
#Информация о собранной модели
model_rnn_mlp_cv.summary()

***
***Важным моментом является - transfer learning и finetuning. Модель обучается постепенно. Сначала модель efficientNet замораживается и обучается голова модели и сети RNN и MLP, затем веса в первой размораживаются на 40% слоев, затем вся модель размораживается до полного обучения. С каждым таким шагом стоит понижать темп обучения. А на первых двух отключить callbacks по преждевременному отключению сети.***
***

***
***Для ускорения расчетов часть с вычисления закомментирована.***
***

In [ ]:
'''
#Заморозка весов
model_cv.trainable = False
#Компиляция задачи: модель, метрика и функция потерь
model_rnn_mlp_cv.compile(
    loss='MAPE', 
    optimizer=O.Adam(lr=0.001), 
    metrics='MAPE'
)
#Обучение
history = model_rnn_mlp_cv.fit(
    trn_dataset.batch(32),
    epochs=100,
    validation_data=val_dataset.batch(32),
    callbacks=create_callbacks_list(100,100,verbose=1),
    verbose=1
)
#График метрик
get_plot_NN(history)
#Загрузка лучшей модели
model_rnn_mlp_cv.load_weights('../working/best_model.hdf5')
#Сохранение модели
model_rnn_mlp_cv.save('../working/MLP_NLP_CV1.hdf5')
'''
print('Удалено.')

In [ ]:
'''
#Обучение модели
model_cv.trainable = True
#Количество слоев
len_lay = len(model_cv.layers)
#Количество слоев для заморозки
fine_tune_lay = int(len_lay * 0.6)
#Заморозка
for layer in model_cv.layers[:fine_tune_lay]:
    layer.trainable = False
#Компиляция задачи: модель, метрика и функция потерь
model_rnn_mlp_cv.compile(
    loss='MAPE', 
    optimizer=O.Adam(lr=0.0001), 
    metrics='MAPE'
)
#Обучение
history = model_rnn_mlp_cv.fit(
    trn_dataset.batch(32),
    epochs=100,
    validation_data=val_dataset.batch(32),
    callbacks=create_callbacks_list(100,100,verbose=1),
    verbose=1
)
#График метрик
get_plot_NN(history)
#Загрузка лучшей модели
model_rnn_mlp_cv.load_weights('../working/best_model.hdf5')
#Сохранение модели
model_rnn_mlp_cv.save('../working/MLP_NLP_CV2.hdf5')
'''
print('Удалено.')

In [ ]:
'''
#Обучение модели
model_cv.trainable = True
#Компиляция задачи: модель, метрика и функция потерь
model_rnn_mlp_cv.compile(
    loss='MAPE', 
    optimizer=O.Adam(lr=0.00001), 
    metrics='MAPE'
)
#Обучение
history = model_rnn_mlp_cv.fit(
    trn_dataset.batch(32),
    epochs=500,
    validation_data=val_dataset.batch(32),
    callbacks=create_callbacks_list(100,75,verbose=1),
    verbose=1
)
#График метрик
get_plot_NN(history)
#Загрузка лучшей модели
model_rnn_mlp_cv.load_weights('../working/best_model.hdf5')
#Сохранение модели
model_rnn_mlp_cv.save('../working/MLP_NLP_CV3.hdf5')
'''
print('Удалено.')

***
***К сожалению, было убито много времени в этот этап и он ничего не дал. Качество на обучающей и валидационной выборках скачет от 10.5% до 12.5%. Метрика пары сабмитов на Kaggle: ~13.6%, ~11.9%. Я могу это объяснить так: скорее всего, либо я выбрал неправильный подход к обучению сразу трех сетей одновременно; либо, как я придумал, можно использовать сети RNN и CNN не как полноценный кусок модели для предсказания, а как лишь правку к уже имеющемуся предсказанию. То есть делать пробросс некого признака, который является предсказанием, полученным ранее, например на CatBoost или MLP. В таком случае сеть уже будет отталкиваться от имеющегося признака. Для полноценных тестов такого решения, к сожалению, не хватило времени.***
***

## 6.4) MLP + Проброс признака (Forwarding)

***
***Последняя попытка улучшить NN сеть - это сделать проброс признака (и Embedding, но его тут нет, так как тесты показали, что он немного ухудшает MAPE на Kaggle).***
***

In [ ]:
#Значимость параметров в модели CatBoostRegressor
s_sub = pd.Series(model_cat.feature_importances_,index=X_trn.columns).sort_values(ascending=False)
s_sub.iloc[:30].plot(kind='bar')
#Выбор количества признаков
n = 6
#Вывод признаков
s_sub.iloc[:n]

***
***Пробрасывать будем самые значимые 6 признаков в модели CatBoost.***
***

***
***Архитектура сети MLP та же.***
***

In [ ]:
#Формирование MLP сети
model_mlp = Sequential(name='MLP')
#Первый слой
model_mlp.add(L.Dense(512, input_dim=X_trn.shape[1], activation='elu'))
model_mlp.add(L.BatchNormalization())
model_mlp.add(L.Dropout(0.35, seed=random_state))
#Второй слой
model_mlp.add(L.Dense(256, activation='elu'))
model_mlp.add(L.BatchNormalization())
model_mlp.add(L.Dropout(0.35, seed=random_state))
#Третий слой
model_mlp.add(L.Dense(128, activation='elu'))
model_mlp.add(L.BatchNormalization())
model_mlp.add(L.Dropout(0.35, seed=random_state))

In [ ]:
#Forwarding
model_for = Sequential(name='Forwarding')
#Первый слой
model_for.add(L.Input(shape=[n]))

In [ ]:
#Группировка сети MLP+For
combinedInput = L.concatenate([model_mlp.output,model_for.output])
#Добавление новых слоев
#Первый слой
head = L.Dense(128, activation='elu')(combinedInput)
head = L.BatchNormalization()(head)
head = L.Dropout(0.35,seed=random_state)(head)
#Второй слой
head = L.Dense(64, activation='elu')(head)
head = L.BatchNormalization()(head)
#Выходной
head = L.Dense(1, activation='linear')(head)
model_mlp_for = M(inputs=[model_mlp.input,model_for.input],outputs=head,name='End')
#Информация о собранной модели
model_mlp_for.summary()

***
***Архитектура головы модели тщательно подбиралась и именно она позволила достичь лучшего результа на Kaggle.***
***

In [ ]:
#Компиляция задачи: модель, метрика и функция потерь
model_mlp_for.compile(
    loss='MAPE', 
    optimizer=O.Adam(lr=0.05), 
    metrics='MAPE'
)
#Обучение
history = model_mlp_for.fit(
    [X_trn,X_trn[s_sub.iloc[:n].index]],Y_trn,
    batch_size=512,
    epochs=1000,
    validation_data=([X_val,X_val[s_sub.iloc[:n].index]],Y_val),
    callbacks=create_callbacks_list(150,120,verbose=1),
    verbose=1
)

In [ ]:
#График метрик
get_plot_NN(history)

In [ ]:
#Загрузка лучшей модели
model_mlp_for.load_weights('../working/best_model.hdf5')
#Сохранение модели
model_mlp_for.save('../working/MLP_FOR.hdf5')

In [ ]:
#Предсказание на train, valid
predict_trn_val(model_mlp_for,[X_trn,X_trn[s_sub.iloc[:n].index]],[X_val, X_val[s_sub.iloc[:n].index]],Y_trn,Y_val)

***
***Точность на обучающей и валидационной выборке примерно такая же, как и в модели MLP.***
***

In [ ]:
#Сохранение на Kaggle
predict_to_Kaggle(model_mlp_for,[test,test[s_sub.iloc[:n].index]],name='submission_for')

***
***Точность на Kaggle данного сабмита (лучшего из многих для данной модели) - 11.588%. 19/55 место. Хуже, чем у классического ML, но лучше, чем у MLP,MLP+RNN,MLP+RNN+CV. Были попытки совместить и RNN и проброс признака, но это ни привело к улучшению результата - метрика на валидации колебалась вокруг одного значения.***
***

# 7.Blend

***
***Модели, основанные на NN, не дали улучшения результата в сравнении с CatBoost, но можно попробовать совместить результат нескольких моделей с помощью blend.***
***

***
***Ниже приведена классификация мною моделей, обученных ранее, по степени качества метрики на Kaggle.***
***

In [ ]:
#Хорошие модели: [cat]+[mlp+for]
#Средние: [ens]+[mlp+rnn]+[xgb]
#Плохие: [mlp+nlp+cv]

***
***Выбираем для blend модели CatBoost и mlp (модель mlp+проброс не выбрана, так как следующим этапом является дообучение модели NN на всей выборке, как CatBoost, и при таком обучении модели с пробросом она резко начинает давать ухудшение метрики.***
***

In [ ]:
#Компиляция задачи: модель, метрика и функция потерь
model_mlp_alone.compile(
    loss='MAPE', 
    optimizer=O.Adam(lr=0.01), 
    metrics='MAPE'
)
#Обучение
history = model_mlp_alone.fit(
    X,Y,
    batch_size=512,
    epochs=1000,
    validation_data=(X,Y),
    callbacks=create_callbacks_list(150,120,verbose=0),
    verbose=0
)
#График метрик
get_plot_NN(history)
#Предсказание на train, valid
predict_trn_val(model_mlp_alone,X_trn,X_val,Y_trn,Y_val)

In [ ]:
#Составляем DF результатов
df_sub_0 = pd.DataFrame(
    {'Y_true':Y.values,
     'Yp_1':current_pred(model_cat.predict(X)),
     'Yp_2':current_pred(model_mlp_alone.predict(X).flatten())
})
df_sub_1 = pd.DataFrame(
    {'Y_true':0,
     'Yp_1':current_pred(model_cat.predict(test)),
     'Yp_2':current_pred(model_mlp_alone.predict(test).flatten())
})
df_sub_0['Kaggle'] = 0
df_sub_1['Kaggle'] = 1
df_sub = pd.concat([df_sub_0,df_sub_1])

***
***Можно просто взять среднее между двумя предсказаниями, а можно обучить мета-модель выбирать с помощью весов лучший результат. Второй вариант дал результат лучше.***
***

In [ ]:
model_meta = LinearRegression(n_jobs=-1)
model_meta.fit(df_sub.query('Kaggle==0').iloc[:,1:],df_sub.query('Kaggle==0').iloc[:,0])
df_sub['Y_pred'] = model_meta.predict(df_sub.iloc[:,1:])

In [ ]:
#Гистограмма целевого признака
sns.histplot(data=df_sub,x='Y_pred',hue='Kaggle',bins=100)

In [ ]:
#Ошибка на обучении
print('Ошибка на обучении: {:.3f}%. На целевом признаке: {:.3f}%.'.\
      format(mape(df_sub.query('Kaggle==0')['Y_true'],df_sub.query('Kaggle==0')['Y_pred']),\
             mape(np.exp(df_sub.query('Kaggle==0')['Y_true']),np.exp(df_sub.query('Kaggle==0')['Y_pred']))))

***
***Прекрасно, ошибка упала на обеих метриках до 1.8%.***
***

In [ ]:
#Правка
Y_pred_test = current_pred(df_sub.query('Kaggle==1')['Y_pred'])
#Пост-обработка - удаление логорифмирования целевого признака
Y_pred_test = np.exp(Y_pred_test)
#Создание DF для вывода
sample_submission = pd.read_csv(PATH + 'sample_submission.csv')
sample_submission['price'] = Y_pred_test
#Вывод в файл
sample_submission.to_csv('submission_blend'+'.csv', index=False)

***
***Точность на Kaggle данного сабмита - 10.542%. 1/55 место. Лучший результат среди всех на данный момент.***
***

# 8.Conclusions

***
#### 1. Зафиксированы навыки работы с *RNN*, *CNN*, *MLP* сетями.
#### 2. Проведен тщательный *feature engineering* и отобраны *значимые признаки*.
#### 3. Выполнен *blend* моделей и получен сабмит с лучшим результатом на Kaggle: **10.542**%.
***

***
#### **Стоит отметить, что в данной работе не выполнено несколько пунктов, что точно хотелось бы выполнить автору работы:**
#### 1. Работа с выбросами числовых признаков - это конечно и так сократит малое количество прецендентов, но вдруг это будет удачным экспериментом.
#### 2. Проброс лучшего предсказания в сеть с RNN и CNN архитектурой, для уточнения результатов по описанию и картинкам.
#### 3. Применение TTA к CV: на это время было, но результат итак для CNN был очень плохой, что казалось, что TTA не сделает его лучше.
#### 4. Применение механизма Attention в RNN не имеет смысла (как я понял), так как на выходе не последовательность, а число, поэтому смысл в применении механизма этого теряется.
#### 5. Применение механизма Transformer: к сожалению, не успел.
***

***
***Лучший результат (submit) на Git-Hub: https://github.com/MirtosSergey/SF_project.***
***